# Creating code for post processing the segmented sheets
* Just gave up on second step segmentation
* Let's try ideas from Gemini!
    * apply_hysteresis_threshold (3D)
    * binary_closing (2D)
    * binary_fill_holes (2D)
    * Invert and Filter (3D)
* might be good to remove holes and modified Frangi filter that enhances surfaceness rather than vesselness

### Create the object to run inferences

In [51]:
from torch.nn.functional import sigmoid
import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence
import sys
import os
import tifffile
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))
import topometrics.leaderboard
import nibabel as nib

case_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images/3545147380.tif"

numpy_array = tifffile.imread(case_path)
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/vol_3545147380.nii.gz"
nii_data = np.transpose(numpy_array, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)


numpy_array = tifffile.imread("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif")
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/gt_3545147380.nii.gz"
nii_data = np.transpose(numpy_array, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)



numpy_array = tifffile.imread("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/3545147380.tif")
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/pred_3545147380.nii.gz"
nii_data = np.transpose(numpy_array, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

In [ ]:
import sys
sys.path.append("../utils")
from infer_class import VesuviusInferer
import os
import json
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))
from torch.nn.functional import sigmoid
import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence
import sys
import os
import tifffile

config_content = {
  "architecture": "STU-Net",
  "patch_size": [128, 128, 128],
  "device": "cuda",
  "batch_size": 1,
  "num_workers": 4,

  "infer_sw_batch_size": 1,
  "blend_mode": "gaussian",
  "deep_supervision": False,
  "activation": False,

  "infer_overlap": 0.5,
  "TTA": True,
  "TH": 0.5,

  "checkpoint_path": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth", 
  "dataset_path_imgs": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images",
  "pred_save_dir": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash"
}


# 🧪 Initialize Inference Object
infer_object = VesuviusInferer(config_content)

print(f"🚀 Starting inference on dataset: {config_content['dataset_path_imgs']}")
print("📋 Using the following configuration:")
for key, value in config_content.items():
    print(f"  🔹 {key}: {value}")


### Select manually a case to run!
input_data = {
    'image': str(case_path),
    'gt': None
}

logits_pred, pred_list = infer_object.infer(
    input=input_data, 
    test=True, 
    threshold_list=[0.5] # CHANGED FROM 0.5
)
infer_object.save_nifti(
    torch_tensor=pred_list[0], 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/2536049117.nii.gz", 
    real_file=None
)

infer_object.save_tiff(
    torch_tensor=pred_list[0], 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/2536049117.tif"
)

Loading weights from: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth
🚀 Starting inference on dataset: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
📋 Using the following configuration:
  🔹 architecture: STU-Net
  🔹 patch_size: [128, 128, 128]
  🔹 device: cuda
  🔹 batch_size: 1
  🔹 num_workers: 4
  🔹 infer_sw_batch_size: 1
  🔹 blend_mode: gaussian
  🔹 deep_supervision: False
  🔹 activation: False
  🔹 infer_overlap: 0.5
  🔹 TTA: True
  🔹 TH: 0.5
  🔹 checkpoint_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth
  🔹 dataset_path_imgs: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
  🔹 pred_save_dir: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash
Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


  0%|          | 0/64 [00:00<?, ?it/s]Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


Doing axes: (3,)


100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


Doing axes: (4,)


100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


Doing axes: (3, 4)


100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


### Create object for testing

In [6]:
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))
import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence
import topometrics.leaderboard

def score_single_tif(
    gt_path,
    pred_path,
    surface_tolerance,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
    ):
    gt: np.ndarray = load_volume(gt_path)
    pr: np.ndarray = load_volume(pred_path)

    #install_dependencies()
    # The import is here to ensure dependencies are loaded first.
    try:
        # Use a standard import now that the path is reliably set.
        import topometrics.leaderboard
    except Exception as err:
        raise HostVisibleError(f'Failed to import topometrics after installation: {err}')

    score_report = topometrics.leaderboard.compute_leaderboard_score(
        predictions=pr,
        labels=gt,
        dims=(0, 1, 2),
        spacing=(1.0, 1.0, 1.0),  # (z, y, x)
        surface_tolerance=surface_tolerance,  # in spacing units
        voi_connectivity=voi_connectivity,
        voi_transform=voi_transform,
        voi_alpha=voi_alpha,
        combine_weights=(topo_weight, surface_dice_weight, voi_weight),  # (Topo, SurfaceDice, VOI)
        fg_threshold=None,  # None => legacy "!= 0"; else uses "x > threshold"
        ignore_label=2,  # voxels with this GT label are ignored
        ignore_mask=None,  # or pass an explicit boolean mask
    )
    
    return {
        'image_score': np.clip(score_report.score, 0.0, 1.0),
        'topo_score': score_report.topo.toposcore,
        'surface_dice': score_report.surface_dice,
        'voi_score': score_report.voi.voi_score,
        'voi_split': score_report.voi.voi_split,
        'voi_merge': score_report.voi.voi_merge
    }

def load_volume(path):
    im = Image.open(path)
    slices = []
    for i, page in enumerate(ImageSequence.Iterator(im)):
        slice_array = np.array(page)
        slices.append(slice_array)
    volume = np.stack(slices, axis=0)
    return volume #(z,y,x)

"""gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/2536049117.tif"
pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/2536049117.tif"



output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_2536049117.tif"

final_volume = split_by_object(pred_path, output_path)

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report"""

'gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/2536049117.tif"\npred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/2536049117.tif"\n\n\n\noutput_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_2536049117.tif"\n\nfinal_volume = split_by_object(pred_path, output_path)\n\nscore_report = score_single_tif(\n    gt_path=gt_path,\n    pred_path=pred_path,\n    surface_tolerance=2.0,\n    voi_connectivity=26,\n    voi_transform=\'one_over_one_plus\',\n    voi_alpha=0.3,\n    topo_weight=0.3,\n    surface_dice_weight=0.35,\n    voi_weight=0.35,\n)\nprint(pred_path)\nscore_report'

In [17]:
# CESoftDiceclDiceLoss_step3_DAsafe/model_epoch_Final
{'image_score': np.float64(0.3789896346704825),
 'topo_score': 0.053815470880591305,
 'surface_dice': 0.4147439170596602,
 'voi_score': 0.6219560641012114,
 'voi_split': 1.078506929732625,
 'voi_merge': 0.9475951146569563}

# CESoftDiceclDiceLoss_step2_DAsafe/model_epoch_Final
{'image_score': np.float64(0.4170240898070778),
 'topo_score': 0.014990347490347491,
 'surface_dice': 0.43251531692557793,
 'voi_score': 0.7461332132457749,
 'voi_split': 0.2754663703842506,
 'voi_merge': 0.858677784428142}

# Real results of th 0.5 (no post-processing) -> Model before (BCE_Tversky_DAsafe/model_epoch_160.pth)
{'image_score': np.float64(0.3875621574520175),
 'topo_score': 0.04236180904522613,
 'surface_dice': 0.3972573748403367,
 'voi_score': 0.6737529529838052,
 'voi_split': 0.7607155566923927,
 'voi_merge': 0.8533629447888303}

{'image_score': np.float64(0.3875621574520175),
 'topo_score': 0.04236180904522613,
 'surface_dice': 0.3972573748403367,
 'voi_score': 0.6737529529838052,
 'voi_split': 0.7607155566923927,
 'voi_merge': 0.8533629447888303}

### Select each individual object, and remove bellow threshold

In [7]:
import tifffile
import numpy as np
from skimage.measure import label
import nibabel as nib
import numpy as np

def split_by_object(input_path, output_path):
    print("Loading image...")
    volume = tifffile.imread(input_path)

    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = volume > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    labeled_volume = label(binary_mask, connectivity=None)

    # 3. Optimize Data Type to save space
    # We check how many objects were found to choose the smallest file size possible.
    num_features = np.max(labeled_volume)
    if num_features < 255:
        final_volume = labeled_volume.astype(np.uint8)
    elif num_features < 65535:
        final_volume = labeled_volume.astype(np.uint16)
    else:
        final_volume = labeled_volume.astype(np.uint32)

    # 4. Save the new TIFF
    print(f"Saving to {output_path}")
    tifffile.imwrite(output_path, final_volume)

    nii_data = np.transpose(binary_mask, (2, 1, 0)).astype(np.uint16)
    nii_data = np.flip(nii_data, axis=0) 
    nii_data = np.flip(nii_data, axis=1)
    output_path_nii = output_path.replace('.tif', '_B.nii.gz')
    nib.save(nib.Nifti1Image(nii_data.astype(np.uint16), np.eye(4)), output_path_nii)

    nii_data = np.transpose(final_volume, (2, 1, 0)).astype(np.uint16)
    nii_data = np.flip(nii_data, axis=0) 
    nii_data = np.flip(nii_data, axis=1)
    output_path_nii = output_path.replace('.tif', '.nii.gz')
    nib.save(nib.Nifti1Image(nii_data.astype(np.uint16), np.eye(4)), output_path_nii)

    print("Done!")
    return final_volume

input_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/2536049117.tif"
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_2536049117.tif"

#final_volume = split_by_object(input_path, output_path)

* A lot of small segmentations without any connection. Removing those!

In [8]:
import tifffile
import numpy as np
from skimage.morphology import remove_small_objects
from skimage.measure import label
import nibabel as nib
import numpy as np

def split_by_object_np(binary_mask):
    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = binary_mask > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    labeled_volume = label(binary_mask, connectivity=None)

    # 3. Optimize Data Type to save space
    # We check how many objects were found to choose the smallest file size possible.
    num_features = np.max(labeled_volume)
    if num_features < 255:
        final_volume = labeled_volume.astype(np.uint8)
    elif num_features < 65535:
        final_volume = labeled_volume.astype(np.uint16)
    else:
        final_volume = labeled_volume.astype(np.uint32)
    return final_volume

def remove_bellow_th(input_path, th, output_path):
    binary_mask = tifffile.imread(input_path)
    labeled_volume = split_by_object_np(binary_mask) 
    
    # 3. Remove Small Objects
    # connectivity=1 ensures we stick to the same face-to-face logic as before
    print(f"Removing objects smaller than {th} voxels...")
    filtered_volume = remove_small_objects(labeled_volume, min_size=th, connectivity=1)

    # Note: remove_small_objects might return a boolean if the input was boolean. 
    # We ensure it stays labeled by multiplying by the original labels (masking).
    if filtered_volume.dtype == bool:
        filtered_volume =  label(filtered_volume)
    num_features  = np.unique(filtered_volume)
    print(f"After filtering, count: {len(num_features)} elements.")
    if output_path!=None:
        # 4. Save
        print(f"Saving to {output_path}")
        tifffile.imwrite(output_path, filtered_volume.astype(np.uint16)) # categorical label

        # Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
        output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
        nii_data = np.transpose(filtered_volume, (2, 1, 0)).astype(np.uint16)
        nii_data = np.flip(nii_data, axis=0) 
        nii_data = np.flip(nii_data, axis=1)
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)
        nii_data[nii_data>0.5] = 1 
        output_path_nii = f"{output_path.replace('.tif', '_B.nii.gz')}"
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)
        filtered_volume_b = np.zeros_like(filtered_volume)
        filtered_volume_b[filtered_volume>0.5] = 1
        output_path = f"{output_path.replace('.tif', '_B.tif')}"
        tifffile.imwrite(f"{output_path}", filtered_volume_b.astype(np.uint16))

        diff_vol = (labeled_volume-filtered_volume).astype(np.uint16)
        nii_data = np.transpose(diff_vol, (2, 1, 0)).astype(np.uint16)
        nii_data = np.flip(nii_data, axis=0) 
        nii_data = np.flip(nii_data, axis=1)
        output_path_nii = f"{output_path.replace('_B.tif', '_diff.nii.gz')}"
        nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)
    print("Done.")
    return filtered_volume, filtered_volume_b

In [12]:
final_volume_th = remove_bellow_th(
    input_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/2536049117.tif", 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117.tif")

pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117_B.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Labeling individual elements...
Found 58 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 20 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117.tif
Done.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117_B.tif


{'image_score': np.float64(0.608705707343169),
 'topo_score': 0.1467789258834035,
 'surface_dice': 0.951426136804701,
 'voi_score': 0.6619225191328646,
 'voi_split': 0.8073663566711468,
 'voi_merge': 0.8951364342900751}

In [ ]:
{'image_score': np.float64(0.6051958352815727),
 'topo_score': 0.1349917081260365,
 'surface_dice': 0.9515386536493884,
 'voi_score': 0.6618851259042164,
 'voi_split': 0.807663674979235,
 'voi_merge': 0.8951236157603629}
print("")

In [ ]:
{'image_score': np.float64(0.4169773941561092),
 'topo_score': 0.015107769885926884, #(a bit better)
 'surface_dice': 0.43201433613258, #(a bit worst)
 'voi_score': 0.7464001301255088, #(a bit better)
 'voi_split': 0.2738130046588314,
 'voi_merge': 0.8587335551059598}

# Removing small objects reduce slightly the topo_score and voi_score, increase sligthly the dice, (OLD)
original_results = {'image_score': np.float64(0.3875621574520175),
 'topo_score': 0.04236180904522613,
 'surface_dice': 0.3972573748403367,
 'voi_score': 0.6737529529838052,
 'voi_split': 0.7607155566923927,
 'voi_merge': 0.8533629447888303}

### Remove the corners (check if the noise created there produces such bad results)

In [5]:
def mask_borders(volume, thickness=5):
    # Create a copy to avoid modifying the original if needed
    cleaned = volume.copy()
    
    # Zero out the faces of the 3D cube
    cleaned[:thickness, :, :] = 0  # Top
    cleaned[-thickness:, :, :] = 0 # Bottom
    cleaned[:, :thickness, :] = 0  # Front
    cleaned[:, -thickness:, :] = 0 # Back
    cleaned[:, :, :thickness] = 0  # Left
    cleaned[:, :, -thickness:] = 0 # Right
    
    return cleaned

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
binary_pred_here = prob_pred>0.5
binary_pred_here = mask_borders(binary_pred_here, thickness=15)
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/corner_removed_small_2536049117.tif"
tifffile.imwrite(output_path, binary_pred_here.astype(np.uint16)) # categorical label


pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/corner_removed_small_2536049117.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/corner_removed_small_2536049117.tif


{'image_score': np.float64(0.7006303641862162),
 'topo_score': 0.5044776119402985,
 'surface_dice': 0.9172864259190723,
 'voi_score': 0.6521052329498606,
 'voi_split': 0.8894989436585066,
 'voi_merge': 0.8888171392452924}

In [11]:
input_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/corner_removed_small_2536049117.tif"
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_corner_removed_small_2536049117.tif"

final_volume = split_by_object(input_path, output_path)

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_corner_removed_small_2536049117.tif
Done!


In [ ]:
# mask_borders(binary_pred_here, thickness=3)
{'image_score': np.float64(0.6075685870946677),
 'topo_score': 0.14337035751475186,
 'surface_dice': 0.951139313542954,
 'voi_score': 0.6618820574291663,
 'voi_split': 0.8086779664586242,
 'voi_merge': 0.8941326716588209}

# mask_borders(binary_pred_here, thickness=5)
{'image_score': np.float64(0.620543871998595),
 'topo_score': 0.1851451091731234,
 'surface_dice': 0.9537665060204878,
 'voi_score': 0.6605201775413919,
 'voi_split': 0.8216025117197475,
 'voi_merge': 0.8915917958446843}

# mask_borders(binary_pred_here, thickness=7)
{'image_score': np.float64(0.6478875382507037),
 'topo_score': 0.28624269954574955,
 'surface_dice': 0.946833278770648,
 'voi_score': 0.6589230880492913,
 'voi_split': 0.8350876632402718,
 'voi_merge': 0.8903383545389787}

# mask_borders(binary_pred_here, thickness=9)
{'image_score': np.float64(0.6922823126894415),
 'topo_score': 0.44346399868787933,
 'surface_dice': 0.9405331110012576,
 'voi_score': 0.6573043549503931,
 'voi_split': 0.8483567493287503,
 'voi_merge': 0.8895273955560556}

# mask_borders(binary_pred_here, thickness=12)
{'image_score': np.float64(0.7046475944805687),
 'topo_score': 0.4997539773659177,
 'surface_dice': 0.93010923173767,
 'voi_score': 0.6548090576074538,
 'voi_split': 0.8684439641439806,
 'voi_merge': 0.8887651967158658}

# mask_borders(binary_pred_here, thickness=15)
{'image_score': np.float64(0.7006303641862162),
 'topo_score': 0.5044776119402985,
 'surface_dice': 0.9172864259190723,
 'voi_score': 0.6521052329498606,
 'voi_split': 0.8894989436585066,
 'voi_merge': 0.8888171392452924}

### Try to fill the holes

In [13]:
from skimage import morphology 
from scipy import ndimage

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
binary_pred_here = prob_pred>0.5

# Removing the small objects
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117.tif"
tifffile.imwrite(output_path, binary_pred_here.astype(np.uint16)) # categorical label
_ = split_by_object(input_path, output_path)

rm_small_2536049117 = load_volume(output_path)

# FILL HOLES (Keep this to fix internal bubbles, but don't rely on it for rips)
mask_holes_filled = morphology.remove_small_holes(
    rm_small_2536049117.astype(bool), 
    area_threshold=1000
)

# 2. ROBUST BINARY CLOSING
# Switch from a 'cross' to a 'ball' to fill diagonal gaps and round holes better.
# Radius 2 closes gaps ~4 pixels wide. Radius 3 closes gaps ~6 pixels wide.
closing_radius = 6
structure_close = morphology.ball(closing_radius)

print(f"Closing with ball radius {closing_radius}...")
mask_closed = ndimage.binary_closing(
    mask_holes_filled, 
    structure=structure_close, 
    iterations=1 # Keep iterations low, rely on radius instead for shape control
)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/rm_holes_2536049117.tif"
tifffile.imwrite(output_path, mask_closed.astype(np.uint16)) # categorical label

input_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/rm_holes_2536049117.tif"
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_rm_holes_2536049117.tif"

final_volume = split_by_object(input_path, output_path)

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/rm_holes_2536049117.tif",
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(output_path)
score_report



Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117.tif
Done!
Closing with ball radius 6...
Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_rm_holes_2536049117.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_rm_holes_2536049117.tif


{'image_score': np.float64(0.7142958290213111),
 'topo_score': 0.5701492537313433,
 'surface_dice': 0.9049234867129121,
 'voi_score': 0.6472223787211105,
 'voi_split': 0.9000569239982591,
 'voi_merge': 0.9168230896170828}

In [ ]:
from skimage import morphology 
from scipy import ndimage
from tqdm import tqdm

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
binary_pred_here = prob_pred>0.5

# Removing the small objects
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/rm_small_2536049117.tif"
tifffile.imwrite(output_path, binary_pred_here.astype(np.uint16)) # categorical label
_ = remove_bellow_th(input_path=output_path, th=1000, output_path=output_path)

rm_small_2536049117 = load_volume(output_path)

# FILL HOLES (Keep this to fix internal bubbles, but don't rely on it for rips)
mask_holes_filled = morphology.remove_small_holes(
    rm_small_2536049117.astype(bool), 
    area_threshold=1000
)

# 2. SPLIT BY OBJECT (Create Categorical Volume)
input_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/filled_holes_2536049117.tif"
categorical_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_filled_holes_2536049117.tif" 

# Save intermediate if needed for split_by_object
tifffile.imwrite(input_path, mask_holes_filled.astype(np.uint16))
_ = split_by_object(input_path, categorical_path)

# ==========================================
# 3. FAST PER-LABEL CLOSING (Bounding Box Optimization)
# ==========================================

print("Loading categorical volume...")
categorical_vol = tifffile.imread(categorical_path)

# Setup: Radius 12 is VERY large (diameter ~25). 
# This requires significant padding to avoid edge artifacts.
closing_radius = 7
structure_close = morphology.ball(closing_radius)

print(f"finding objects...")
# find_objects returns a list of slices (bounding boxes) for labels 1, 2, 3...
# This is the key to the speedup.
object_slices = ndimage.find_objects(categorical_vol)

final_closed_vol = np.zeros_like(categorical_vol)

print(f"Processing {len(object_slices)} objects with Bounding Box optimization...")

for idx, sl in enumerate(tqdm(object_slices)):
    if sl is None: continue # Label was skipped/missing
    
    label_id = idx + 1
    
    # A. EXPAND SLICE (PADDING)
    # We must pad the bounding box by the closing radius, otherwise
    # the closing operation will hit the edge of the crop and fail.
    z_sl, y_sl, x_sl = sl
    
    # Pad and clip to volume boundaries
    z_min = max(0, z_sl.start - closing_radius)
    z_max = min(categorical_vol.shape[0], z_sl.stop + closing_radius)
    y_min = max(0, y_sl.start - closing_radius)
    y_max = min(categorical_vol.shape[1], y_sl.stop + closing_radius)
    x_min = max(0, x_sl.start - closing_radius)
    x_max = min(categorical_vol.shape[2], x_sl.stop + closing_radius)
    
    padded_slice = (slice(z_min, z_max), slice(y_min, y_max), slice(x_min, x_max))
    
    # B. CROP
    vol_crop = categorical_vol[padded_slice]
    obj_mask_crop = (vol_crop == label_id)
    
    # C. CLOSE (Fast because volume is small)
    obj_closed_crop = ndimage.binary_closing(
        obj_mask_crop, 
        structure=structure_close, 
        iterations=1
    )
    
    # D. PASTE BACK
    # We only update the specific region in the final volume
    # Use simple boolean indexing on the crop
    
    # Get the current crop from the final volume to handle overlaps safely
    final_crop = final_closed_vol[padded_slice]
    
    # "Last label wins" strategy (simple overwrite)
    # Only write where we actually have a closed object
    final_crop[obj_closed_crop] = label_id
    
    # Assign back to main volume
    final_closed_vol[padded_slice] = final_crop

# ==========================================
# 4. SAVE AND SCORE
# ==========================================

output_path_final = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/rm_holes_2536049117.tif"
tifffile.imwrite(output_path_final, final_closed_vol.astype(np.uint16))
print(f"Saved closed volume to: {output_path_final}")

_ = split_by_object(output_path_final, "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_rm_holes_2536049117.tif")

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=output_path_final,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report




Removing objects smaller than 1000 voxels...
After filtering, count: 20 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/rm_small_2536049117.tif
Done.
Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_4/categorical_filled_holes_2536049117.tif
Done!
Loading categorical volume...
finding objects...
Processing 19 objects with Bounding Box optimization...


 37%|███▋      | 7/19 [01:02<02:05, 10.47s/it]

In [ ]:
from scipy.ndimage import generate_binary_structure, binary_opening
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/1013184726.tif"
pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/1013184726.tif"
binary_pred_here = load_volume(pred_path)

# Removing the small objects
rm_small_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_1013184726.tif"
tifffile.imwrite(rm_small_path, binary_pred_here.astype(np.uint16)) # categorical label
filtered_volume, filtered_volume_b = remove_bellow_th(
    input_path=rm_small_path, 
    th=1000, 
    output_path=rm_small_path)

rm_small_path_B = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_1013184726_B.tif"

# Binary opening
rm_bridges = binary_opening(filtered_volume_b, structure=np.ones((3,1,3)))
rm_bridges_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_bridges_1013184726.tif"
tifffile.imwrite(rm_bridges_path, rm_bridges.astype(np.uint16)) # categorical label

rm_bridges_rm_small_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.tif"
filtered_volume, filtered_volume_b = remove_bellow_th(
    input_path=rm_bridges_path, 
    th=1000, 
    output_path=rm_bridges_rm_small_path)

rm_bridges_rm_small_path_B = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726_B.tif"

# Save intermediate if needed for split_by_object
_ = split_by_object(rm_bridges_rm_small_path_B, rm_bridges_rm_small_path)
print(rm_bridges_rm_small_path.replace('.tif', '.nii.gz'))


score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=rm_bridges_rm_small_path_B,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report


Labeling individual elements...
Found 112 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 24 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_1013184726.tif
Done.
Labeling individual elements...
Found 54 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 24 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.tif
Done.
Loading image...
Labeling individual elements...
Initial count: 23 elements.
Found 23 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.nii.gz


{'image_score': np.float64(0.5322698397359252),
 'topo_score': 0.2670856245090338,
 'surface_dice': 0.7428282548700696,
 'voi_score': 0.5490121805105447,
 'voi_split': 1.6716041120374625,
 'voi_merge': 1.0665732636148524}

In [3]:
from scipy.ndimage import binary_opening

def erode_2d(vol, s):
    # assume shape = (z, y, x)
    # xy-plane
    vol_xy = np.stack([binary_opening(vol[i, :, :], structure=np.ones((s, s))) for i in range(vol.shape[0])], axis=0)

    # xz-plane
    vol_xz = np.stack([binary_opening(vol[:, j, :], structure=np.ones((s, s))) for j in range(vol.shape[1])], axis=1)

    # yz-plane
    vol_yz = np.stack([binary_opening(vol[:, :, k], structure=np.ones((s, s))) for k in range(vol.shape[2])], axis=2)

    # combine results
    rm_bridges = vol_xy | vol_xz | vol_yz
    return rm_bridges

In [ ]:
rm_small_path_B = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_1013184726_B.tif"
rm_bridges=erode_2d(vol=load_volume(rm_small_path_B), s=3)
rm_bridges_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_bridges_1013184726.tif"
tifffile.imwrite(rm_bridges_path, rm_bridges.astype(np.uint16)) # categorical label

rm_bridges_rm_small_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.tif"
filtered_volume, filtered_volume_b = remove_bellow_th(
    input_path=rm_bridges_path, 
    th=1000, 
    output_path=rm_bridges_rm_small_path)

rm_bridges_rm_small_path_B = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726_B.tif"

_ = split_by_object(rm_bridges_rm_small_path_B, rm_bridges_rm_small_path)
print(rm_bridges_rm_small_path.replace('.tif', '.nii.gz'))



score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=rm_bridges_rm_small_path_B,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report

# erosion (3) -> 22 elements
# erosion (5) -> 38 elements

Labeling individual elements...
Found 50 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 23 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.tif
Done.
Loading image...
Labeling individual elements...
Initial count: 22 elements.
Found 22 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_bridges_1013184726.nii.gz


{'image_score': np.float64(0.5367764802419565),
 'topo_score': 0.2819237147595357,
 'surface_dice': 0.7427706250347151,
 'voi_score': 0.5492275630055584,
 'voi_split': 1.6704664476229587,
 'voi_merge': 1.0653299529780524}

In [ ]:
# Original
{'image_score': np.float64(0.4882821996410088),
 'topo_score': 0.12257552815631985,
 'surface_dice': 0.7409740257701427,
 'voi_score': 0.5490532347844651,
 'voi_split': 1.675570628474354,
 'voi_merge': 1.0621527630322238}

# in the next cell (2D erosion and 3D fill holes)
{'image_score': np.float64(0.5193702573700877),
 'topo_score': 0.27159974774017237,
 'surface_dice': 0.7009397715603349,
 'voi_score': 0.5501754657197676,
 'voi_split': 1.6384501603188388,
 'voi_merge': 1.0868896770471015}

# # in the next cell (2D erosion and 3D fill holes) -> less agressive (3,3,7)
{'image_score': np.float64(0.5266403974739573),
 'topo_score': 0.2664179104477612,
 'surface_dice': 0.730360307830454,
 'voi_score': 0.5459683331399141,
 'voi_split': 1.6721598269374063,
 'voi_merge': 1.0998670428806532}

# 2D (3) erosion
{'image_score': np.float64(0.5367764802419565),
 'topo_score': 0.2819237147595357,
 'surface_dice': 0.7427706250347151,
 'voi_score': 0.5492275630055584,
 'voi_split': 1.6704664476229587,
 'voi_merge': 1.0653299529780524}

# 2d (1) erosion
{'image_score': np.float64(0.4955838446400916),
 'topo_score': 0.1466655163488914,
 'surface_dice': 0.7410665837104861,
 'voi_score': 0.5491739583907256,
 'voi_split': 1.6741187853307682,
 'voi_merge': 1.062270020202755}

## just some erosion
{'image_score': np.float64(0.5322698397359252),
 'topo_score': 0.2670856245090338,
 'surface_dice': 0.7428282548700696,
 'voi_score': 0.5490121805105447,
 'voi_split': 1.6716041120374625,
 'voi_merge': 1.0665732636148524}

In [62]:
# FIXED CODE, GOOD RESULTS!
import numpy as np
import tifffile
import cupy as cp
import cupyx.scipy.ndimage as ndimage_gpu
from skimage import morphology 
from skimage.measure import regionprops, label
from tqdm import tqdm

#prob_pred = sigmoid(logits_pred)
#prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
#binary_pred_here = prob_pred>0.5
binary_pred_here = load_volume("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/3515764944.tif")

# Removing the small objects
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117.tif"
tifffile.imwrite(output_path, binary_pred_here.astype(np.uint16)) # categorical label
_ = remove_bellow_th(input_path=output_path, th=1000, output_path=output_path)

rm_small_2536049117 = load_volume(output_path)

# FILL HOLES (Keep this to fix internal bubbles, but don't rely on it for rips)
mask_holes_filled = morphology.remove_small_holes(
    rm_small_2536049117.astype(bool), 
    area_threshold=1000
)

# Remove bridges # added LATER!
# Didn't change the results for the case 2536049117!
mask_holes_filled = erode_2d(vol=mask_holes_filled, s=3)

# 2. SPLIT BY OBJECT (Create Categorical Volume)
input_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/filled_holes_2536049117.tif"
categorical_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_filled_holes_2536049117.tif" 

# Save intermediate if needed for split_by_object
tifffile.imwrite(input_path, mask_holes_filled.astype(np.uint16))
_ = split_by_object(input_path, categorical_path)


# Load your labels (CPU is fine for loading)
labels = tifffile.imread(categorical_path)
n_labels = labels.max()
print(f"Number of sheets: {n_labels}")

# Prepare output
dtype_out = np.uint32 if n_labels > np.iinfo(np.uint16).max else np.uint16
filled_labels = np.zeros_like(labels, dtype=dtype_out)

# ==========================================================
# TUNING FOR "STRAIGHT PAPER" LOOK
# ==========================================================
# Balanced Dilation/Erosion = No thickening, just filling.
DILATION_RADIUS = 5 # 5
EROSION_RADIUS = 5   # 5
# Median filter "irons" the sheet flat and straightens edges
MEDIAN_FILTER_SIZE = 3 # 3 
# Remove edge artifacts from U-Net
CROP_MARGIN = 13 # 13      
# Padding to prevent "invisible wall" squeezing
PAD_WIDTH = DILATION_RADIUS + 4

# ==========================================================
# GPU SETUP: PRE-LOAD KERNELS
# ==========================================================
print("Creating kernels and moving to GPU...")
# 1. Create balls on CPU
cpu_ball_dilate = morphology.ball(DILATION_RADIUS)
cpu_ball_erode = morphology.ball(EROSION_RADIUS)

# 2. Move to GPU (do this ONCE, not in the loop)
gpu_ball_dilate = cp.array(cpu_ball_dilate)
gpu_ball_erode = cp.array(cpu_ball_erode)

print(f"Processing {n_labels} sheets on CUDA 13.0 GPU...")

for region in tqdm(regionprops(labels), total=n_labels):

    label_id = region.label

    # --- CPU PART (Slicing is fast on CPU) ---
    bbox = region.bbox
    minr, minc, mins, maxr, maxc, maxs = bbox
    slices = (slice(minr, maxr), slice(minc, maxc), slice(mins, maxs))

    # Extract binary sub-volume
    sub = (labels[slices] == label_id)
    
    # 1. Pre-Cleaning (Remove artifacts)
    if CROP_MARGIN > 0:
        sub[:CROP_MARGIN, :, :] = 0
        sub[-CROP_MARGIN:, :, :] = 0
        sub[:, :CROP_MARGIN, :] = 0
        sub[:, -CROP_MARGIN:, :] = 0
        sub[:, :, :CROP_MARGIN] = 0
        sub[:, :, -CROP_MARGIN:] = 0

    # 2. Pad (CPU)
    sub_padded = np.pad(sub, PAD_WIDTH, mode='constant', constant_values=0)

    # --- GPU PART (Heavy lifting) ---
    # 3. Transfer to VRAM
    gpu_sub = cp.array(sub_padded)

    # 4. Solidify (Dilation + Erosion)
    # Note: We use 'structure=' to pass our pre-loaded ball
    gpu_sub = ndimage_gpu.binary_dilation(gpu_sub, structure=gpu_ball_dilate)
    gpu_sub = ndimage_gpu.binary_erosion(gpu_sub, structure=gpu_ball_erode)

    # 5. Fill Internal Holes
    gpu_sub = ndimage_gpu.binary_fill_holes(gpu_sub)

    # 6. Ironing / Straightening (Median Filter)
    if MEDIAN_FILTER_SIZE > 0:
        gpu_sub = ndimage_gpu.median_filter(gpu_sub, size=MEDIAN_FILTER_SIZE)

    # --- RETRIEVE PART ---
    # 7. Pull back to CPU
    sub_filled = gpu_sub.get()

    # 8. Un-pad
    sub_final = sub_filled[PAD_WIDTH:-PAD_WIDTH, PAD_WIDTH:-PAD_WIDTH, PAD_WIDTH:-PAD_WIDTH]

    # 9. Write back
    out_block = filled_labels[slices]
    out_block[sub_final] = label_id
    filled_labels[slices] = out_block

# ==========================================
# 4. SAVE AND SCORE
# ==========================================

output_path_final = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_holes_2536049117.tif"
tifffile.imwrite(output_path_final, filled_labels.astype(np.uint16))
print(f"Saved closed volume to: {output_path_final}")

_ = split_by_object(output_path_final, "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_holes_2536049117.tif")

gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3515764944.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=output_path_final,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report

Labeling individual elements...
Found 73 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 15 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_small_2536049117.tif
Done.
Loading image...
Labeling individual elements...
Initial count: 21 elements.
Found 21 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_filled_holes_2536049117.tif
Done!
Number of sheets: 21
Creating kernels and moving to GPU...
Processing 21 sheets on CUDA 13.0 GPU...


100%|██████████| 21/21 [00:07<00:00,  2.85it/s]


Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/rm_holes_2536049117.tif
Loading image...
Labeling individual elements...
Initial count: 12 elements.
Found 12 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_holes_2536049117.tif
Done!


{'image_score': np.float64(0.7882637960836423),
 'topo_score': 0.9743589743589743,
 'surface_dice': 0.8558458386594815,
 'voi_score': 0.5611716007003755,
 'voi_split': 1.3514921274294143,
 'voi_merge': 1.2551282524564094}

In [ ]:
# Create a function with the cell before, looks very nice the results!
# FIXED CODE, GOOD RESULTS!
import numpy as np
import tifffile
import cupy as cp
import cupyx.scipy.ndimage as ndimage_gpu
from skimage import morphology 
from skimage.measure import regionprops, label
from tqdm import self.config['post_process_hysteresis_low_th']tqdm


def post_process(previous_pred_path, output_path):
    binary_pred_here = load_volume(previous_pred_path)
    # Remove small objects
    
    labeled_volume = split_by_object_np(binary_pred_here) 
    filtered_volume = remove_small_objects(labeled_volume, min_size=1000, connectivity=1)
    
    # FILL HOLES (Keep this to fix internal bubbles, but don't rely on it for rips)
    mask_holes_filled = morphology.remove_small_holes(
        split_by_object_np(filtered_volume).astype(bool), 
        area_threshold=1000
    )

    # Remove bridges
    # Didn't change the results for the case 2536049117!
    mask_holes_filled = erode_2d(vol=mask_holes_filled, s=3)
    labeled_volume = split_by_object_np(mask_holes_filled) 

    n_labels = labeled_volume.max()

    # Prepare output
    dtype_out = np.uint32 if n_labels > np.iinfo(np.uint16).max else np.uint16
    filled_labels = np.zeros_like(labeled_volume, dtype=dtype_out)

    # ==========================================================
    # TUNING FOR "STRAIGHT PAPER" LOOK
    # ==========================================================
    # Balanced Dilation/Erosion = No thickening, just filling.
    DILATION_RADIUS = 5 # 5
    EROSION_RADIUS = 5   # 5
    # Median filter "irons" the sheet flat and straightens edges
    MEDIAN_FILTER_SIZE = 3 # 3 
    # Remove edge artifacts from U-Net
    CROP_MARGIN = 13 # 13      
    # Padding to prevent "invisible wall" squeezing
    PAD_WIDTH = DILATION_RADIUS + 4

    # ==========================================================
    # GPU SETUP: PRE-LOAD KERNELS
    # ==========================================================

    # 1. Create balls on CPU
    cpu_ball_dilate = morphology.ball(DILATION_RADIUS)
    cpu_ball_erode = morphology.ball(EROSION_RADIUS)

    # 2. Move to GPU (do this ONCE, not in the loop)
    gpu_ball_dilate = cp.array(cpu_ball_dilate)
    gpu_ball_erode = cp.array(cpu_ball_erode)

    case_name = previous_pred_path.split('/')[-1]

    for region in tqdm(regionprops(labeled_volume), total=n_labels, desc=f"Processing {case_name}"):

        label_id = region.label

        # --- CPU PART (Slicing is fast on CPU) ---
        bbox = region.bbox
        minr, minc, mins, maxr, maxc, maxs = bbox
        slices = (slice(minr, maxr), slice(minc, maxc), slice(mins, maxs))

        # Extract binary sub-volume
        sub = (labeled_volume[slices] == label_id)
        
        # 1. Pre-Cleaning (Remove artifacts)
        if CROP_MARGIN > 0:
            sub[:CROP_MARGIN, :, :] = 0
            sub[-CROP_MARGIN:, :, :] = 0
            sub[:, :CROP_MARGIN, :] = 0
            sub[:, -CROP_MARGIN:, :] = 0
            sub[:, :, :CROP_MARGIN] = 0
            sub[:, :, -CROP_MARGIN:] = 0

        # 2. Pad (CPU)
        sub_padded = np.pad(sub, PAD_WIDTH, mode='constant', constant_values=0)

        # --- GPU PART (Heavy lifting) ---
        # 3. Transfer to VRAM
        gpu_sub = cp.array(sub_padded)

        # 4. Solidify (Dilation + Erosion)
        # Note: We use 'structure=' to pass our pre-loaded ball
        gpu_sub = ndimage_gpu.binary_dilation(gpu_sub, structure=gpu_ball_dilate)
        gpu_sub = ndimage_gpu.binary_erosion(gpu_sub, structure=gpu_ball_erode)

        # 5. Fill Internal Holes
        gpu_sub = ndimage_gpu.binary_fill_holes(gpu_sub)

        # 6. Ironing / Straightening (Median Filter)
        if MEDIAN_FILTER_SIZE > 0:
            gpu_sub = ndimage_gpu.median_filter(gpu_sub, size=MEDIAN_FILTER_SIZE)

        # --- RETRIEVE PART ---
        # 7. Pull back to CPU
        sub_filled = gpu_sub.get()

        # 8. Un-pad
        sub_final = sub_filled[PAD_WIDTH:-PAD_WIDTH, PAD_WIDTH:-PAD_WIDTH, PAD_WIDTH:-PAD_WIDTH]

        # 9. Write back
        out_block = filled_labels[slices]
        out_block[sub_final] = label_id
        filled_labels[slices] = out_block

    # ==========================================
    # 4. SAVE AND SCORE
    # ==========================================
    
    output_path_final = os.path.join(output_path, case_name)
    tifffile.imwrite(output_path, filled_labels.astype(np.uint16))
    print(f"Saved closed volume to: {output_path}")


root_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5"
for file_name in os.listdir(root_path):
    if not file_name.endswith('.tif'):
        continue
    output_path=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/{file_name}"
    if os.path.exists(output_path):
      print("Path exists!")  
    else:
        post_process(
            previous_pred_path=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/{file_name}",
            output_path=output_path
            )


Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!
Path exists!


Processing 114235076.tif: 100%|██████████| 53/53 [00:41<00:00,  1.29it/s]


Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/114235076.tif


Processing 3527825464.tif: 100%|██████████| 27/27 [00:30<00:00,  1.11s/it]


Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/3527825464.tif


...

Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/2821927657.tif


Processing 149409101.tif: 100%|██████████| 22/22 [00:40<00:00,  1.82s/it]


Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/149409101.tif


Processing 3686818985.tif: 100%|██████████| 74/74 [00:40<00:00,  1.82it/s]

Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/3686818985.tif


In [18]:
import sys
sys.path.append("../utils")
from infer_class import VesuviusInferer
from compute_metrics_obj import VesuviusMetric
def create_dfs(path_dir):
    # Generate DataFrame
    result_df = glob.glob(os.path.join(path_dir, '**/*.tif'), recursive=True)
    result_df = pd.DataFrame({'tif_paths': result_df})
    result_df['id'] = result_df['tif_paths'].apply(lambda x: os.path.basename(x).split('.')[0])

    # save dataframe to csv (take name from the path_dir last folder)
    csv_name = os.path.basename(os.path.normpath(path_dir)) + '_df.csv'
    result_df.to_csv(os.path.join(path_dir, csv_name), index=False)
    return result_df

create_dfs("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/")

test_metric_obj = VesuviusMetric(
    solution_path=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/val_labels_df.csv",
    submission_path=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/th_0.5_df.csv",
    output_file=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/results_post.csv"
)

test_metric_obj._run()

  0%|          | 0/79 [00:00<?, ?it/s]

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/1669670049.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/1669670049.tif


  3%|▎         | 2/79 [00:20<12:57, 10.09s/it]

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/541673255.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/541673255.tif


  4%|▍         | 3/79 [00:51<24:00, 18.96s/it]

...

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/149409101.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/149409101.tif


100%|██████████| 79/79 [43:26<00:00, 28.76s/it]

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3686818985.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/3686818985.tif


100%|██████████| 79/79 [43:56<00:00, 33.37s/it]


📊 FINAL AGGREGATED METRICS
  id  image_score  topo_score  surface_dice  voi_score  voi_split  voi_merge
MEAN     0.542944    0.325685      0.726135   0.545976    1.29212   1.512489
✅ Saved detailed metrics with Mean row to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/results_post.csv


In [21]:

# case 3515764944 with 2D erode
{'image_score': np.float64(0.7882637960836423),
 'topo_score': 0.9743589743589743,
 'surface_dice': 0.8558458386594815,
 'voi_score': 0.5611716007003755,
 'voi_split': 1.3514921274294143,
 'voi_merge': 1.2551282524564094}

# Original 2536049117
{'image_score': np.float64(0.6051958352815727),
 'topo_score': 0.1349917081260365,
 'surface_dice': 0.9515386536493884,
 'voi_score': 0.6618851259042164,
 'voi_split': 0.807663674979235,
 'voi_merge': 0.8951236157603629}

KeyboardInterrupt: 

#### apply_hysteresis_threshold (3D)

In [ ]:
from skimage.filters import apply_hysteresis_threshold
# Original case without any post-process
"""
input_data = {
    'image': str("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images/114235076.tif"),
    'gt': None
}
logits_pred, pred = infer_object.infer(
    input=input_data, 
    test=True, 
    threshold=0.5 # CHANGED FROM 0.5
)
"""
# The predictions from MONAI are transposed
# Need to change back!
prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))



Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


  0%|          | 0/64 [00:00<?, ?it/s]Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


Doing axes: (3,)


100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


Doing axes: (4,)


100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


Doing axes: (3, 4)


100%|██████████| 64/64 [00:43<00:00,  1.46it/s]


NameError: name 'np' is not defined

In [14]:
binary_pred_here = prob_pred>0.5
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/pred_bin_114235076_original_step2.tif"
tifffile.imwrite(output_path, binary_pred_here.astype(np.uint16)) # categorical label

In [ ]:
# strong = 0.8 (High confidence)
# weak = 0.4 (Low confidence to fill gaps)
from skimage.filters import apply_hysteresis_threshold
hysteresis_th0408_mask = apply_hysteresis_threshold(prob_pred, 0.3, 0.75)
prob_pred.shape

(320, 320, 320)

In [16]:
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/hysteresisth0306_114235076_original_step2.tif"
tifffile.imwrite(output_path, hysteresis_th0408_mask.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
nii_data = np.transpose(hysteresis_th0408_mask, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/hysteresisth0306_114235076_original_step2.tif


{'image_score': np.float64(0.42506195607724895),
 'topo_score': 0.022449367503441044,
 'surface_dice': 0.4498825537293728,
 'voi_score': 0.7453378629169602,
 'voi_split': 0.2776122491090761,
 'voi_merge': 0.8612991527008893}

In [ ]:


# apply_hysteresis_threshold(prob_pred, 0.3, 0.8) / apply_hysteresis_threshold(prob_pred, 0.3, 0.7) / apply_hysteresis_threshold(prob_pred, 0.3, 0.75)
{'image_score': np.float64(0.4250627816266543),
 'topo_score': 0.022449367503441044,
 'surface_dice': 0.44988506234095993,
 'voi_score': 0.7453377130179598,
 'voi_split': 0.2776121850446437,
 'voi_merge': 0.8613001162032592}

# apply_hysteresis_threshold(prob_pred, 0.5, 0.8)
{'image_score': np.float64(0.4164614636310076),
 'topo_score': 0.01589706132155597,
 'surface_dice': 0.42982216008774154,
 'voi_score': 0.7464416834395179,
 'voi_split': 0.27298504675551283,
 'voi_merge': 0.8593129040239527}

# Original
{'image_score': np.float64(0.417026590342083),
 'topo_score': 0.014997131209421713,
 'surface_dice': 0.43251717612714397,
 'voi_score': 0.7461326838135888,
 'voi_split': 0.27546518298702793,
 'voi_merge': 0.858682141806286}

In [ ]:
# OLD
#(prob_pred, 0.1, 0.6)/(prob_pred, 0.1, 0.5) -> 3 º best topo_score ⬇️⬇️ | 1º best surface_dice ⬆️⬆️ | 1º best voi_score ⬆️⬆️
{'image_score': np.float64(0.43291155605791964),
 'topo_score': 0.01021792298480274,
 'surface_dice': 0.4877857312370883,
 'voi_score': 0.7403462092271368,
 'voi_split': 0.2942298236737297,
 'voi_merge': 0.8748348991848787}


# (prob_pred, 0.4, 0.6)/(prob_pred, 0.4, 0.5) -> 2º best topo_score ⬇️ | 2º best surface_dice ⬆️ | 1º best voi_score ⬆️⬆️
{'image_score': np.float64(0.4139674579625019),
 'topo_score': 0.028642246642246644,
 'surface_dice': 0.4152043704381358,
 'voi_score': 0.7430092980470867,
 'voi_split': 0.26621442121472194,
 'voi_merge': 0.8867128374722746}


# Real results of th 0.5 (no post-processing)
{'image_score': np.float64(0.3875621574520175),
 'topo_score': 0.04236180904522613,
 'surface_dice': 0.3972573748403367,
 'voi_score': 0.6737529529838052,
 'voi_split': 0.7607155566923927,
 'voi_merge': 0.8533629447888303}

In [17]:
rm_small_obj_hysteresisth0406_114235076_original = remove_bellow_th(
    input_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/hysteresisth0306_114235076_original_step2.tif", 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_hysteresisth0306_114235076_original_step2.tif"
    )

pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_hysteresisth0306_114235076_original_step2.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Labeling individual elements...
Found 10 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 10 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_hysteresisth0306_114235076_original_step2.tif
Done.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_hysteresisth0306_114235076_original_step2.tif


{'image_score': np.float64(0.42506195607724895),
 'topo_score': 0.022449367503441044,
 'surface_dice': 0.4498825537293728,
 'voi_score': 0.7453378629169602,
 'voi_split': 0.2776122491090761,
 'voi_merge': 0.8612991527008893}

#### morphological opening (remove the bridges)

In [ ]:
# improves topo score, but dice and voi-score are similar to the baseline 
# file after hysteresisth0406
from scipy.ndimage import binary_opening

# 1. Run your current 3D Hysteresis
hysteresisth0306_114235076_original = apply_hysteresis_threshold(prob_pred, 0.3, 0.75)

# 2. Break the bridges
# We use a small kernel. This erodes 1 pixel (snapping thin bridges) 
# and then dilates back (restoring the sheet volume).
opened_hysteresisth0306_114235076_original = binary_opening(hysteresisth0306_114235076_original, structure=np.ones((3,3,3)))


output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_hysteresisth0306_114235076_original_step2.tif"
tifffile.imwrite(output_path, opened_hysteresisth0306_114235076_original.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
nii_data = np.transpose(opened_hysteresisth0306_114235076_original, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)


pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Loading image...
Labeling individual elements...
Initial count: 23 elements.
Found 23 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_hysteresisth0306_114235076_original_step2categorical.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_hysteresisth0306_114235076_original_step2categorical.tif
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_hysteresisth0306_114235076_original_step2.tif


{'image_score': np.float64(0.42960935551974644),
 'topo_score': 0.041951640759930915,
 'surface_dice': 0.44682589262753275,
 'voi_score': 0.7446708596346588,
 'voi_split': 0.28223171545206566,
 'voi_merge': 0.8606854855361559}

In [ ]:
# apply_hysteresis_threshold(prob_pred, 0.3, 0.75)
{'image_score': np.float64(0.42506195607724895),
 'topo_score': 0.022449367503441044,
 'surface_dice': 0.4498825537293728,
 'voi_score': 0.7453378629169602,
 'voi_split': 0.2776122491090761,
 'voi_merge': 0.8612991527008893}

# morphological open
{'image_score': np.float64(0.42960935551974644),
 'topo_score': 0.041951640759930915,
 'surface_dice': 0.44682589262753275,
 'voi_score': 0.7446708596346588,
 'voi_split': 0.28223171545206566,
 'voi_merge': 0.8606854855361559}

In [ ]:
# Old
# only apply_hysteresis_threshold(prob_pred, 0.4, 0.6)/(prob_pred, 0.4, 0.5) 
{'image_score': np.float64(0.4139674579625019),
 'topo_score': 0.028642246642246644,
 'surface_dice': 0.4152043704381358,
 'voi_score': 0.7430092980470867,
 'voi_split': 0.26621442121472194,
 'voi_merge': 0.8867128374722746}

# apply_hysteresis_threshold with binary opening -> 1º best topo_score ⬆️ | 3º best surface_dice --  | 3º best voi_score ⬇️
{'image_score': np.float64(0.4080548261067526),
 'topo_score': 0.11740136054421768,
 'surface_dice': 0.3921057543415669,
 'voi_score': 0.6731354397826823,
 'voi_split': 0.784066827086829,
 'voi_merge': 0.8345502738185694}

# Real results of th 0.5 (no post-processing)
{'image_score': np.float64(0.3875621574520175),
 'topo_score': 0.04236180904522613,
 'surface_dice': 0.3972573748403367,
 'voi_score': 0.6737529529838052,
 'voi_split': 0.7607155566923927,
 'voi_merge': 0.8533629447888303}

# Didn't improve anything!

In [ ]:
# Don't result, the results are similar to not using any post-processing

import numpy as np
from scipy.ndimage import binary_opening

def safe_2d_opening(mask):
    # Standard 3x3 square for 2D slices
    kernel_2d = np.ones((3,3), dtype=bool)
    
    result = np.zeros_like(mask)
    # Process slice by slice
    for z in range(mask.shape[0]):
        result[z] = binary_opening(mask[z], structure=kernel_2d)
    
    return result

# Apply
hysteresisth0306_114235076_original = apply_hysteresis_threshold(prob_pred, 0.4, 0.6)
safe_2d_open_hysteresisth0306_114235076_original = safe_2d_opening(hysteresisth0306_114235076_original)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/safe_2d_open_hysteresisth0306_114235076_original.tif"
tifffile.imwrite(output_path, safe_2d_open_hysteresisth0306_114235076_original.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
nii_data = np.transpose(safe_2d_open_hysteresisth0306_114235076_original, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/safe_2d_open_hysteresisth0306_114235076_original.tif


{'image_score': np.float64(0.38833766422225846),
 'topo_score': 0.04299961195188203,
 'surface_dice': 0.39831017053128825,
 'voi_score': 0.6743692027164084,
 'voi_split': 0.7782432074585012,
 'voi_merge': 0.8313142672498488}

In [ ]:
import numpy as np
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import label, distance_transform_edt
from skimage.filters import apply_hysteresis_threshold

def watershed_rescue(prob_map, core_thresh=0.75, mask_low=0.4, mask_high=0.6):
    """
    Uses Watershed to cut thick bridges between sheets without eroding the sheets themselves.
    """
    # 1. The Mask (Your Hysteresis Result)
    # This defines the total volume we want to keep.
    mask = apply_hysteresis_threshold(prob_map, mask_low, mask_high)
    
    # 2. The Markers (High Confidence Cores)
    # These are the "safe" centers of the sheets.
    # We label them 1, 2, 3... so they are distinct basins.
    core_mask = prob_map > core_thresh
    markers, _ = label(core_mask)
    
    # 3. The Landscape (Distance Transform)
    # We want to grow from the center of the sheet to the edge.
    # Using distance transform ensures we grow geometrically.
    distance = distance_transform_edt(mask)
    
    # 4. Run Watershed
    # -distance makes the "peaks" (centers) into "valleys" for the water to fill.
    # It fills the 'mask' starting from 'markers'.
    w_labels = watershed(-distance, markers, mask=mask)
    
    # 5. The "Dam" (Boundary Removal)
    # Watershed returns touching labels (e.g., Label 1 touches Label 2).
    # To fix Topology, we must physically separate them with 0s.
    boundaries = find_boundaries(w_labels, mode='thick')
    
    # Remove boundaries from the original mask
    final_mask = mask.copy()
    final_mask[boundaries] = 0
    
    return final_mask.astype(np.uint8)

# --- Apply it ---
# Adjust core_thresh if your model is very confident (try 0.8) or unsure (try 0.7)
watershed_mask = watershed_rescue(prob_pred, core_thresh=0.75, mask_low=0.4, mask_high=0.6)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/watershed_hysteresisth0306_114235076_original.tif"
tifffile.imwrite(output_path, watershed_mask.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
nii_data = np.transpose(watershed_mask, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/watershed_hysteresisth0306_114235076_original.tif


{'image_score': np.float64(0.3611867062647005),
 'topo_score': 0.00477387287457351,
 'surface_dice': 0.3542049603708464,
 'voi_score': 0.673665166492949,
 'voi_split': 0.6674853456891917,
 'voi_merge': 0.9472378617162813}

In [49]:
# watershed_rescue(prob_pred, core_thresh=0.75, mask_low=0.4, mask_high=0.6)
{'image_score': np.float64(0.36099775268716866),
 'topo_score': 0.004588647391893799,
 'surface_dice': 0.3549358963738418,
 'voi_score': 0.6725531278250167,
 'voi_split': 0.6729972145660972,
 'voi_merge': 0.949907398852922}

#watershed_rescue(prob_pred, core_thresh=0.8, mask_low=0.4, mask_high=0.6)
{'image_score': np.float64(0.3599932381733456),
 'topo_score': 0.0041923316201568205,
 'surface_dice': 0.35443929413312997,
 'voi_score': 0.6705193878305802,
 'voi_split': 0.6832966715990239,
 'voi_merge': 0.9546406157697626}

{'image_score': np.float64(0.3599932381733456),
 'topo_score': 0.0041923316201568205,
 'surface_dice': 0.35443929413312997,
 'voi_score': 0.6705193878305802,
 'voi_split': 0.6832966715990239,
 'voi_merge': 0.9546406157697626}

In [ ]:
from skimage.measure import label
rm_small_obj_hysteresisth0406_114235076_original = remove_bellow_th(
    input_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/watershed_hysteresisth0306_114235076_original.tif", 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_watershed_hysteresisth0306_114235076_original.tif"
    )

Labeling individual elements...
Found 650 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 19 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/rm_small_watershed_hysteresisth0306_114235076_original.tif
Done.


In [63]:
import numpy as np
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import label, distance_transform_edt, binary_fill_holes, binary_closing
from skimage.filters import apply_hysteresis_threshold, gaussian
from skimage.morphology import remove_small_objects

def watershed_rescue_v4(prob_map, sigma=1.0, core_thresh=0.75, min_seed_size=1000):
    """
    v4 Improvements:
    1. Aggressive Size Filter (1000+ voxels).
    2. Seed Fusion: Merges split cores using binary closing.
    """
    
    # --- Step 1: The Mask (Boundaries) ---
    mask = apply_hysteresis_threshold(prob_map, 0.4, 0.6)
    
    # --- Step 2: The Cores ---
    smoothed_prob = gaussian(prob_map, sigma=sigma)
    core_mask = smoothed_prob > core_thresh
    
    # --- Step 3: FUSION (The Fix) ---
    # Close gaps between nearby seed fragments.
    # A 3x3x3 kernel connects fragments within 1-2 voxels.
    # Iterations=2 extends this reach to ~4 voxels.
    core_mask = binary_closing(core_mask, structure=np.ones((3,3,3)), iterations=2)
    
    # --- Step 4: Aggressive Filtering ---
    # Real sheets are big. Dust is small.
    # Try 500 first, then 1000 if you still have >20 seeds.
    core_mask = remove_small_objects(core_mask, min_size=min_seed_size)
    
    # Label the fused, large seeds
    markers, num_markers = label(core_mask)
    print(f"Found {num_markers} unique sheets (seeds) after Fusion & Filtering.")

    # --- Step 5: Watershed ---
    distance = distance_transform_edt(mask)
    w_labels = watershed(-distance, markers, mask=mask)
    
    # --- Step 6: Cut & Heal ---
    boundaries = find_boundaries(w_labels, mode='thick')
    
    final_mask = mask.copy()
    final_mask[boundaries] = 0
    
    # 2D Heal (Slice-by-slice)
    for z in range(final_mask.shape[0]):
        final_mask[z] = binary_fill_holes(final_mask[z])

    return final_mask.astype(np.uint8)

In [ ]:
# --- Apply it ---
# Adjust core_thresh if your model is very confident (try 0.8) or unsure (try 0.7)
watershed_mask = watershed_rescue_v4(prob_pred,  sigma=1.0, core_thresh=0.75, min_seed_size=1000)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/watershedV4_hysteresisth0306_114235076_original.tif"
tifffile.imwrite(output_path, watershed_mask.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
nii_data = np.transpose(watershed_mask, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Found 28 unique sheets (seeds) after Fusion & Filtering.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/watershedV4_hysteresisth0306_114235076_original.tif


{'image_score': np.float64(0.36191352061581733),
 'topo_score': 0.005427408412483039,
 'surface_dice': 0.35545616561823895,
 'voi_score': 0.6739304003591107,
 'voi_split': 0.6641997843201765,
 'voi_merge': 0.948576052606702}

In [ ]:
from skimage.measure import label
rm_small_obj_hysteresisth0406_114235076_original = remove_bellow_th(
    input_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/watershedV4_hysteresisth0306_114235076_original.tif", 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_watershedV4_hysteresisth0306_114235076_original.tif"
    )

Labeling individual elements...
Found 280 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 18 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/rm_small_watershedV4_hysteresisth0306_114235076_original.tif
Done.


In [ ]:
from scipy.ndimage import generate_binary_structure, binary_opening
from skimage.measure import label

prob_pred_th75 = prob_pred>0.75
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/core_114235076_original.tif"
tifffile.imwrite(output_path, prob_pred_th75.astype(np.uint16)) # categorical label

cross_kernel = generate_binary_structure(rank=3, connectivity=1)
opened_prob_pred_th75 = binary_opening(prob_pred_th75, structure=cross_kernel)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_core_114235076_original.tif"
tifffile.imwrite(output_path, opened_prob_pred_th75.astype(np.uint16)) # categorical label

rm_small_opened_core_114235076_original = remove_bellow_th(
    input_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_core_114235076_original.tif", 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_opened_core_114235076_original.tif"
    )

pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/opened_core_114235076_original.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Labeling individual elements...
Found 225 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 23 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/rm_small_opened_core_114235076_original.tif
Done.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/opened_core_114235076_original.tif


{'image_score': np.float64(0.33917118749368297),
 'topo_score': 0.009390211624989932,
 'surface_dice': 0.32175708696170763,
 'voi_score': 0.6392546959131095,
 'voi_split': 0.9309669363801343,
 'voi_merge': 0.9501054353115218}

In [ ]:
# This gives similar results as using only apply_hysteresis_threshold(prob_pred, 0.4, 0.6) 
import numpy as np
from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt, binary_fill_holes
from skimage.filters import apply_hysteresis_threshold
import tifffile
from skimage.measure import label

import numpy as np
from skimage.segmentation import watershed

import numpy as np
from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt
from skimage.measure import label
from scipy.ndimage import distance_transform_edt, binary_erosion
def grow_seeds_back(prob_pred, seeds_mask, target_threshold=0.4):
    
    # 1. Prepare Markers (Your 23 seeds)
    if seeds_mask.max() > 1:
        print("Using existing labels from input file.")
        markers = seeds_mask
    else:
        print("Input is binary. Generating unique labels...")
        markers, num_features = label(seeds_mask > 0, connectivity=2) 
        print(f"Found {num_features} unique seeds.")
    
    num_features  = np.unique(markers)
    print(f"Using as seeds {len(num_features)} elemets.")
    
    # 2. Prepare Mask (The Territory)
    mask = prob_pred > target_threshold

    # 3. Distance Transform
    distance = distance_transform_edt(mask)

    # 4. Watershed with Separation
    final_labels = watershed(-distance, markers, mask=mask, watershed_line=True)
    
    # --- CRITICAL FIX: Safety Erosion ---
    # We erode each object to ensure NO diagonal connections exist.
    # We do this on the binary mask of the labels.
    #print("Applying Safety Erosion to guarantee separation...")
    #binary_mask = final_labels > 0
    #eroded_mask = binary_erosion(binary_mask)
    
    # Apply the erosion to the labels
    #final_labels = final_labels * eroded_mask 
    
    return final_labels

# Run the growth
grow_seeds_back_opened_prob_pred_th75 = grow_seeds_back(prob_pred, rm_small_opened_core_114235076_original, target_threshold=0.4)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/grow_seeds_back_opened_prob_pred_th75.tif"
tifffile.imwrite(output_path, grow_seeds_back_opened_prob_pred_th75.astype(np.uint16)) # categorical label

_ = remove_bellow_th(
    input_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/grow_seeds_back_opened_prob_pred_th75.tif", 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/rm_small_grow_seeds_back_opened_prob_pred_th75.tif"
    )

pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/grow_seeds_back_opened_prob_pred_th75.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report


Using existing labels from input file.
Using as seeds 23 elemets.
Labeling individual elements...
Found 7 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 8 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/rm_small_grow_seeds_back_opened_prob_pred_th75.tif
Done.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/grow_seeds_back_opened_prob_pred_th75.tif


{'image_score': np.float64(0.41388934583166737),
 'topo_score': 0.029472527472527474,
 'surface_dice': 0.4138934281930054,
 'voi_score': 0.7433853934924491,
 'voi_split': 0.2634887168440685,
 'voi_merge': 0.8871688413751251}

In [ ]:
# Real results of th 0.5 (no post-processing)
{'image_score': np.float64(0.3875621574520175),
 'topo_score': 0.04236180904522613,
 'surface_dice': 0.3972573748403367,
 'voi_score': 0.6737529529838052,
 'voi_split': 0.7607155566923927,
 'voi_merge': 0.8533629447888303}

# only apply_hysteresis_threshold(prob_pred, 0.4, 0.6) 
{'image_score': np.float64(0.4139674579625019),
 'topo_score': 0.028642246642246644,
 'surface_dice': 0.4152043704381358,
 'voi_score': 0.7430092980470867,
 'voi_split': 0.26621442121472194,
 'voi_merge': 0.8867128374722746}

In [ ]:
import numpy as np
import tifffile
from scipy.ndimage import distance_transform_edt, binary_fill_holes
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import remove_small_objects, binary_closing, disk
from skimage.filters import apply_hysteresis_threshold

def final_safe_pipeline(prob_map, core_th=0.70, mask_low=0.4, mask_high=0.6):
    
    print("--- Step 1: Generate & Fuse Seeds (2D) ---")
    # 1. Base Cores
    cores = prob_map > core_th
    
    # 2. ANISOTROPIC FUSION (Fixes Splits)
    # We close gaps ONLY in X-Y planes to fix 'broken' sheets without bridging Z-gaps.
    fused_cores = np.zeros_like(cores)
    search_kernel = disk(4) # Radius 4 covers gaps up to ~8 pixels
    
    for z in range(cores.shape[0]):
        fused_cores[z] = binary_closing(cores[z], footprint=search_kernel)
        
    # 3. Filter Dust
    # Only keep massive seeds (>2000 voxels)
    clean_seeds = remove_small_objects(fused_cores, min_size=2000)
    
    # 4. Label Seeds
    markers, num_seeds = label(clean_seeds, connectivity=2, return_num=True)
    print(f"Found {num_seeds} unique sheets.")

    print("--- Step 2: Watershed Growth ---")
    # 5. Define Mask
    mask = apply_hysteresis_threshold(prob_map, mask_low, mask_high)
    
    # 6. Distance Map
    distance = distance_transform_edt(mask)
    
    # 7. Watershed
    # watershed_line=True attempts to leave a gap
    labeled_volume = watershed(-distance, markers, mask=mask, watershed_line=True)
    
    print("--- Step 3: SAFE 2D Hole Filling ---")
    # CRITICAL FIX: We must NOT overwrite existing labels.
    
    # Get IDs
    ids = np.unique(labeled_volume)
    ids = ids[ids > 0]
    
    for i in ids:
        sheet_mask = (labeled_volume == i)
        
        for z in range(sheet_mask.shape[0]):
            if np.any(sheet_mask[z]):
                # Calculate the filled version
                filled_slice = binary_fill_holes(sheet_mask[z])
                
                # BUG FIX: Only fill where the volume is currently EMPTY (0)
                # If there is another label there, DO NOT touch it.
                # This prevents Sheet 1 from eating Sheet 2.
                safe_fill_locations = filled_slice & (labeled_volume[z] == 0)
                
                labeled_volume[z, safe_fill_locations] = i

    print("--- Step 4: Surgical Boundary Cut (Anti-Merge) ---")
    # Even with watershed_line, diagonal pixels can touch in 3D.
    # We find pixels that sit exactly between two DIFFERENT labels and delete them.
    
    # mode='thick' finds the boundary pixels on both sides
    # background=0 ignores boundaries between Sheet and Air (we keep those)
    boundaries = find_boundaries(labeled_volume, mode='thick', background=0)
    
    labeled_volume[boundaries] = 0

    return labeled_volume.astype(np.uint16)

# --- EXECUTION ---
# 1. Run Pipeline
final_result = final_safe_pipeline(prob_pred, core_th=0.70)

# 2. Save
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_safe_fix.tif"
tifffile.imwrite(output_path, final_result)

# 3. Filter & Score
_ = remove_bellow_th(
    input_path=output_path, 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_submission.tif"
)

pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_safe_fix.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

--- Step 1: Generate & Fuse Seeds (2D) ---
Found 10 unique sheets.
--- Step 2: Watershed Growth ---
--- Step 3: SAFE 2D Hole Filling ---
--- Step 4: Surgical Boundary Cut (Anti-Merge) ---
Labeling individual elements...
Found 246 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 14 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/final_submission.tif
Done.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/final_safe_fix.tif


{'image_score': np.float64(0.36029888949372785),
 'topo_score': 0.005149291593781473,
 'surface_dice': 0.3532960955239081,
 'voi_score': 0.6717156245206444,
 'voi_split': 0.6801008883603463,
 'voi_merge': 0.9489832235043475}

In [ ]:
import numpy as np
import tifffile
from scipy.ndimage import binary_fill_holes
from skimage.morphology import binary_closing, disk

def close_holes_instance_wise(labeled_volume):
    """
    Closes holes inside each sheet individually to improve TopoScore (b1).
    Prevents merging by checking for collisions with other sheets.
    """
    print("--- Starting Hole Closing ---")
    
    # Create a fresh volume for the result
    final_volume = np.zeros_like(labeled_volume)
    
    # Get all unique sheet IDs (skip 0)
    unique_ids = np.unique(labeled_volume)
    unique_ids = unique_ids[unique_ids > 0] 
    
    print(f"Processing {len(unique_ids)} sheets individually...")
    
    for i in unique_ids:
        # 1. Isolate the Sheet (Binary Mask)
        sheet_mask = (labeled_volume == i)
        
        # 2. Heal the Sheet (Slice-by-Slice 2D)
        healed_mask = np.zeros_like(sheet_mask)
        
        for z in range(sheet_mask.shape[0]):
            if np.any(sheet_mask[z]):
                # A. Close small cracks (Fuses fragments)
                # disk(3) = Radius 3 (approx 7x7 kernel). 
                # Good for closing gaps up to ~6 pixels.
                closed_slice = binary_closing(sheet_mask[z], footprint=disk(3))
                
                # B. Fill Enclosed Holes (The 'Swiss Cheese' Fix)
                # Fills any black region completely surrounded by white
                filled_slice = binary_fill_holes(closed_slice)
                
                healed_mask[z] = filled_slice
        
        # 3. The "Anti-Cannibalism" Check (CRITICAL)
        # If growing Sheet A accidentally covers Sheet B, we must block it.
        # Collision = Where our new mask overlaps with a DIFFERENT label in original
        collision_mask = (labeled_volume != 0) & (labeled_volume != i)
        
        # Delete pixels that would cause a merge
        healed_mask[collision_mask] = 0
        
        # 4. Add to Final Volume
        final_volume[healed_mask] = i
        
    return final_volume

# 1. Load your Watershed result (The one that had good separation but holes)
# This should be the result BEFORE you ran 'find_boundaries'
# e.g. the output of 'watershed(..., watershed_line=True)'
input_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_safe_fix.tif"
labeled_volume = tifffile.imread(input_path)

# 2. Run the Polish
final_closed_volume = close_holes_instance_wise(labeled_volume)

# 3. Save
output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_closed_volume.tif"
tifffile.imwrite(output_path, final_closed_volume.astype(np.uint16))

# 4. Filter Small Objects & Score
_ = remove_bellow_th(
    input_path=output_path, 
    th=1000, 
    output_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/categorical_final_closed_volume.tif"
)


pred_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_closed_volume.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

--- Starting Hole Closing ---
Processing 10 sheets individually...
Labeling individual elements...
Found 191 individual elements.
Removing objects smaller than 1000 voxels...
After filtering, count: 13 elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/categorical_final_closed_volume.tif
Done.
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process/final_closed_volume.tif


{'image_score': np.float64(0.35968116171561904),
 'topo_score': 0.0046960109803786156,
 'surface_dice': 0.35274151747157173,
 'voi_score': 0.6708937923041581,
 'voi_split': 0.6849528995627435,
 'voi_merge': 0.9502100798885929}

# New try

In [22]:
import numpy as np
from skimage import morphology
from scipy import ndimage

# 0. Base Hysteresis (Your 0.4/0.6 baseline)
hysteresis_mask = apply_hysteresis_threshold(prob_pred, 0.3, 0.75)

# 1. FILL HOLES (Fixes Topo k=2 Cavities)
# Safe. Fills internal bubbles.
mask_holes_filled = morphology.remove_small_holes(
    hysteresis_mask.astype(bool), 
    area_threshold=100
)

# 2. BINARY CLOSING (Fixes Topo k=1 Loops & Improves VOI)
# Instead of smoothing (which cuts), we close (which connects).
# This fills tiny loops (fixing topology) and reinforces thin bridges.
# We use a small ball/cross structure.
structure_close = ndimage.generate_binary_structure(3, 1) # connectivity 1 (cross)
mask_closed = ndimage.binary_closing(
    mask_holes_filled, 
    structure=structure_close, 
    iterations=1
)

# 3. REMOVE DUST (Fixes Topo k=0 Components)
# lowered min_size to 100 to be safer. 
# Only remove things that are clearly noise, not valid fragments.
final_mask = morphology.remove_small_objects(
    mask_closed, 
    min_size=100
)

# Prepare for save...

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.tif"
tifffile.imwrite(output_path, final_mask.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
print(output_path_nii)
nii_data = np.transpose(final_mask, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

_ = split_by_object(input_path=output_path, output_path=output_path.replace('.tif', 'categorical.tif'))
print(output_path.replace('.tif', 'categorical.tif'))

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

NameError: name 'apply_hysteresis_threshold' is not defined

In [23]:
# apply_hysteresis_threshold(prob_pred, 0.3, 0.75)
{'image_score': np.float64(0.42506195607724895),
 'topo_score': 0.022449367503441044,
 'surface_dice': 0.4498825537293728,
 'voi_score': 0.7453378629169602,
 'voi_split': 0.2776122491090761,
 'voi_merge': 0.8612991527008893}

{'image_score': np.float64(0.42506195607724895),
 'topo_score': 0.022449367503441044,
 'surface_dice': 0.4498825537293728,
 'voi_score': 0.7453378629169602,
 'voi_split': 0.2776122491090761,
 'voi_merge': 0.8612991527008893}

In [30]:
import numpy as np
from skimage import morphology
from scipy import ndimage
import tqdm

# 0. Base Hysteresis (Your 0.4/0.6 baseline)
hysteresis_mask = apply_hysteresis_threshold(prob_pred, 0.3, 0.75)

# 1. FILL HOLES (Fixes Topo k=2 Cavities)
# Safe. Fills internal bubbles.
mask_holes_filled = morphology.remove_small_holes(
    hysteresis_mask.astype(bool), 
    area_threshold=100
)

# 2. BINARY CLOSING (Fixes Topo k=1 Loops & Improves VOI)
# Instead of smoothing (which cuts), we close (which connects).
# This fills tiny loops (fixing topology) and reinforces thin bridges.
# We use a small ball/cross structure.
structure_close = ndimage.generate_binary_structure(3, 1) # connectivity 1 (cross)
mask_closed = ndimage.binary_closing(
    mask_holes_filled, 
    structure=structure_close, 
    iterations=1
)

# 3. REMOVE DUST (Fixes Topo k=0 Components)
# lowered min_size to 100 to be safer. 
# Only remove things that are clearly noise, not valid fragments.
mask_solid = morphology.remove_small_objects(
    mask_closed, 
    min_size=100
)

final_mask = np.zeros_like(mask_solid)
BRIDGE_SIZE_LIMIT = 150  # Any 2D blob smaller than this is considered a bridge/noise.

print("Removing bridges slice-by-slice...")
for z in tqdm.tqdm(range(mask_solid.shape[0])):
    slice_2d = mask_solid[z, :, :]
    
    # If the slice is empty, skip
    if not np.any(slice_2d):
        continue
        
    # Remove small objects in 2D (Snips the vertical bridges)
    slice_clean = morphology.remove_small_objects(
        slice_2d.astype(bool), 
        min_size=BRIDGE_SIZE_LIMIT
    )
    
    final_mask[z, :, :] = slice_clean

# --- STEP 4: FINAL 3D DUSTING (Fix Topo k=0) ---
# Remove any floating islands that survived the process
final_mask = morphology.remove_small_objects(
    final_mask.astype(bool), 
    min_size=200
)

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.tif"
tifffile.imwrite(output_path, final_mask.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
print(output_path_nii)
nii_data = np.transpose(final_mask, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

_ = split_by_object(input_path=output_path, output_path=output_path.replace('.tif', 'categorical.tif'))
print(output_path.replace('.tif', 'categorical.tif'))

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Removing bridges slice-by-slice...


100%|██████████| 320/320 [00:00<00:00, 1221.11it/s]


/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.nii.gz
Loading image...
Labeling individual elements...
Initial count: 6 elements.
Found 6 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2categorical.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2categorical.tif
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.tif


{'image_score': np.float64(0.4387828224445737),
 'topo_score': 0.07122407170294495,
 'surface_dice': 0.4473556263224002,
 'voi_score': 0.7452603763452857,
 'voi_split': 0.27857698955038457,
 'voi_merge': 0.8607994026484641}

In [48]:
import numpy as np
from skimage.feature import hessian_matrix, hessian_matrix_eigvals

def modified_frangi_surface_3d_debug(volume, scale_range=(1, 3), scale_step=1, alpha=0.5, beta=0.5):
    """
    Debug version of the Modified Frangi Filter.
    """
    sigmas = np.arange(scale_range[0], scale_range[1], scale_step)
    filtered_max = np.zeros_like(volume)
    
    # DEBUG: Check input volume stats
    print(f"--- Frangi Input Stats ---")
    print(f"Shape: {volume.shape}")
    print(f"Min: {volume.min():.4f}, Max: {volume.max():.4f}, Mean: {volume.mean():.4f}")
    print(f"Non-zero voxels: {np.count_nonzero(volume)}")
    
    if np.max(volume) == 0:
        print("!!! ERROR: Input volume is all zeros. Filter will return zeros.")
        return filtered_max

    for sigma in sigmas:
        print(f"\nProcessing scale sigma={sigma}...")
        
        # 1. Compute Hessian & Eigenvalues
        H_elems = hessian_matrix(volume, sigma=sigma, order='rc')
        eigvals = hessian_matrix_eigvals(H_elems)
        
        # 2. Sort by Absolute Magnitude
        abs_eig = np.abs(eigvals)
        abs_eig = np.sort(abs_eig, axis=0)
        
        lambda1 = abs_eig[0] # Smallest (Noise/Plane-1)
        lambda2 = abs_eig[1] # Middle   (Plane-2)
        lambda3 = abs_eig[2] # Largest  (Normal to surface)
        
        # 3. Compute Ratios
        Ra = lambda2 / (lambda3 + 1e-15)
        Rb = lambda1 / (np.sqrt(lambda2 * lambda3) + 1e-15)
        S = np.sqrt(np.sum(eigvals**2, axis=0))
        
        # DEBUG: Check intermediate metrics
        max_S = np.max(S)
        print(f"  Max Structureness (S): {max_S:.4f}")
        print(f"  Mean Ra (Plate-ness): {np.mean(Ra):.4f} (Target: Low)")
        
        if max_S == 0:
            print("  !!! S is zero everywhere. Image likely constant or empty.")
            continue

        # 4. Dynamic Gamma (Crucial for binary masks)
        # We set gamma based on the max structure found in this scale
        gamma = max_S / 2.0
        
        # 5. Compute Filter Response
        term_Ra = np.exp(-(Ra**2) / (2 * alpha**2))
        term_Rb = np.exp(-(Rb**2) / (2 * beta**2))
        term_S  = (1 - np.exp(-(S**2) / (2 * gamma**2)))
        
        response = term_Ra * term_Rb * term_S
        
        print(f"  Max Response for this scale: {np.max(response):.4f}")
        
        filtered_max = np.maximum(filtered_max, response)
        
    return filtered_max

def apply_post_processing_debug(prob_pred, threshold=0.75):
    print(f"--- Post-Processing Start ---")
    print(f"Input Probabilities - Max: {prob_pred.max():.4f}, Mean: {prob_pred.mean():.4f}")
    
    # 1. CHECK THRESHOLD
    if prob_pred.max() < threshold:
        print(f"!!! WARNING: Max probability ({prob_pred.max():.4f}) is LESS than threshold ({threshold}).")
        print("!!! The mask will be empty. Returning zeros.")
        # OPTION: Auto-adjust threshold for debugging
        # threshold = prob_pred.max() * 0.9
        # print(f"!!! Auto-adjusted threshold to {threshold} for testing.")
        return np.zeros_like(prob_pred)

    # 2. Create Mask
    mask = (prob_pred > threshold).astype(np.float32)
    
    # 3. Run Filter
    # We use scale_range=(1, 2) and step=1 (only sigma=1) first to be safe and fast
    surfaceiness = modified_frangi_surface_3d_debug(mask, scale_range=(1, 3))
    
    return surfaceiness

# RUN IT
# final_output = 

In [49]:
# 1. Run the debug/filter function
surfaceness_map = apply_post_processing_debug(prob_pred, threshold=0.75)

print(f"Surfaceness Max: {surfaceness_map.max()}") 
# If this prints ~0.86, your data is there! It just needs formatting.

# 2. Normalize for Visualization (Optional, helps you see it)
# Scales 0.0-0.86 to 0-255
debug_image = (surfaceness_map / surfaceness_map.max() * 255).astype(np.uint8)

# 3. Create Final Binary Mask (The actual segmentation)
# We threshold the surfaceness score. 
# Since Frangi suppresses noise effectively, a low threshold like 0.1 usually works well.
final_binary_mask = (surfaceness_map > 0.1).astype(np.uint8)

print(f"Final Mask Non-zeros: {final_binary_mask.sum()}")

--- Post-Processing Start ---
Input Probabilities - Max: 0.9990, Mean: 0.2335
--- Frangi Input Stats ---
Shape: (320, 320, 320)
Min: 0.0000, Max: 1.0000, Mean: 0.1666
Non-zero voxels: 5459220

Processing scale sigma=1...


use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current behavior


  Max Structureness (S): 0.3872
  Mean Ra (Plate-ness): 0.0791 (Target: Low)
  Max Response for this scale: 0.8609

Processing scale sigma=2...
  Max Structureness (S): 0.1151
  Mean Ra (Plate-ness): 0.0812 (Target: Low)
  Max Response for this scale: 0.8633
Surfaceness Max: 0.8632922172546387
Final Mask Non-zeros: 14884127


In [50]:
final_mask = final_binary_mask

In [51]:

output_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.tif"
tifffile.imwrite(output_path, final_mask.astype(np.uint16)) # categorical label

# Save as .nii.gz (Nibabel handles the gzipping automatically based on the extension)
output_path_nii = f"{output_path.replace('.tif', '.nii.gz')}"
print(output_path_nii)
nii_data = np.transpose(final_mask, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

_ = split_by_object(input_path=output_path, output_path=output_path.replace('.tif', 'categorical.tif'))
print(output_path.replace('.tif', 'categorical.tif'))

pred_path = output_path
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=pred_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.nii.gz
Loading image...
Labeling individual elements...
Initial count: 42 elements.
Found 42 individual elements.
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2categorical.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2categorical.tif
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_2/final_mask_step2.tif


{'image_score': np.float64(0.4319570491142923),
 'topo_score': 0.0022264114750084347,
 'surface_dice': 0.467906312364647,
 'voi_score': 0.764348332411895,
 'voi_split': 0.20391522700797338,
 'voi_merge': 0.8237648572127385}

# Trying to do the Frangi filter

In [2]:
import sys
sys.path.append("../utils")
from infer_class import VesuviusInferer
import os
import json
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))

case_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images/1013184726.tif"

config_content = {
  "architecture": "STU-Net",
  "patch_size": [128, 128, 128],
  "device": "cuda",
  "batch_size": 1,
  "num_workers": 4,

  "infer_sw_batch_size": 1,
  "blend_mode": "gaussian",
  "deep_supervision": False,
  "activation": False,

  "infer_overlap": 0.5,
  "TTA": True,
  "TH": 0.5,

  "checkpoint_path": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth", 
  "dataset_path_imgs": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images",
  "pred_save_dir": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash"
}


# 🧪 Initialize Inference Object
infer_object = VesuviusInferer(config_content)

print(f"🚀 Starting inference on dataset: {config_content['dataset_path_imgs']}")
print("📋 Using the following configuration:")
for key, value in config_content.items():
    print(f"  🔹 {key}: {value}")


### Select manually a case to run!
input_data = {
    'image': str(case_path),
    'gt': None
}

logits_pred, pred_list = infer_object.infer(
    input=input_data, 
    test=True, 
    threshold_list=[0.5] # CHANGED FROM 0.5
)
infer_object.save_nifti(
    torch_tensor=pred_list[0], 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/1013184726.nii.gz", 
    real_file=None
)

infer_object.save_tiff(
    torch_tensor=pred_list[0], 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/1013184726.tif"
)

Loading weights from: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth
🚀 Starting inference on dataset: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
📋 Using the following configuration:
  🔹 architecture: STU-Net
  🔹 patch_size: [128, 128, 128]
  🔹 device: cuda
  🔹 batch_size: 1
  🔹 num_workers: 4
  🔹 infer_sw_batch_size: 1
  🔹 blend_mode: gaussian
  🔹 deep_supervision: False
  🔹 activation: False
  🔹 infer_overlap: 0.5
  🔹 TTA: True
  🔹 TH: 0.5
  🔹 checkpoint_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth
  🔹 dataset_path_imgs: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
  🔹 pred_save_dir: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash
Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


100%|██████████| 64/64 [01:09<00:00,  1.09s/it]


Doing axes: (3,)


100%|██████████| 64/64 [01:09<00:00,  1.09s/it]


Doing axes: (4,)


100%|██████████| 64/64 [01:09<00:00,  1.08s/it]


Doing axes: (3, 4)


100%|██████████| 64/64 [01:09<00:00,  1.08s/it]


In [20]:
print("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/1013184726.tif")

/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/1013184726.tif


In [7]:
import torch
import torch.nn.functional as F
import math

def _gaussian_1d(sigma, device, dtype):
    radius = int(3 * sigma)
    x = torch.arange(-radius, radius + 1, device=device, dtype=dtype)
    gauss = torch.exp(-x**2 / (2 * sigma**2))
    gauss = gauss / gauss.sum()
    d1 = -x / (sigma**2) * gauss
    d2 = (x**2 - sigma**2) / (sigma**4) * gauss
    return gauss, d1, d2


def _conv_sep_3d(volume, kx, ky, kz):
    volume = F.conv3d(volume, kx[None, None, :, None, None], padding=(kx.numel()//2, 0, 0))
    volume = F.conv3d(volume, ky[None, None, None, :, None], padding=(0, ky.numel()//2, 0))
    volume = F.conv3d(volume, kz[None, None, None, None, :], padding=(0, 0, kz.numel()//2))
    return volume


def _hessian_3d_torch(volume, sigma):
    device = volume.device
    dtype = volume.dtype
    g, g1, g2 = _gaussian_1d(sigma, device, dtype)
    Hxx = _conv_sep_3d(volume, g2, g, g)
    Hyy = _conv_sep_3d(volume, g, g2, g)
    Hzz = _conv_sep_3d(volume, g, g, g2)
    Hxy = _conv_sep_3d(volume, g1, g1, g)
    Hxz = _conv_sep_3d(volume, g1, g, g1)
    Hyz = _conv_sep_3d(volume, g, g1, g1)
    s2 = sigma ** 2
    return Hxx * s2, Hyy * s2, Hzz * s2, Hxy * s2, Hxz * s2, Hyz * s2


def frangi_surfaceness_3d_torch(
    volume,
    sigmas=(0.5, 1.0, 1.5, 2.0),
    beta=0.5,
    gamma=None,
    bright_surface=True,
    eps=1e-12
):
    # Ensure shape = (1,1,D,H,W)
    if volume.ndim == 3:
        volume = volume.unsqueeze(0).unsqueeze(0)
    elif volume.ndim == 4:
        volume = volume.unsqueeze(0)
    elif volume.ndim != 5:
        raise ValueError("volume must be 3D, 4D, or 5D tensor")

    volume = volume.float()
    device = volume.device
    output = torch.zeros_like(volume)

    # sanitize input
    volume = torch.nan_to_num(volume, nan=0.0, posinf=1.0, neginf=0.0)
    volume = volume.clamp(0.0, 1.0)

    for sigma in sigmas:
        sigma = float(sigma)
        Hxx, Hyy, Hzz, Hxy, Hxz, Hyz = _hessian_3d_torch(volume, sigma)

        # Build Hessian
        H = torch.stack([
            torch.stack([Hxx, Hxy, Hxz], dim=-1),
            torch.stack([Hxy, Hyy, Hyz], dim=-1),
            torch.stack([Hxz, Hyz, Hzz], dim=-1)
        ], dim=-2)

        # Clamp Hessian to remove NaNs/Infs
        H = torch.nan_to_num(H, nan=0.0, posinf=1e6, neginf=-1e6)

        # Compute eigenvalues safely on CPU
        eigvals = torch.linalg.eigvalsh(H.cpu()).to(device)

        # Sort by absolute value
        idx = torch.argsort(eigvals.abs(), dim=-1)
        eigvals = torch.gather(eigvals, -1, idx)

        l1 = eigvals[..., 0]
        l2 = eigvals[..., 1]
        l3 = eigvals[..., 2]

        valid = l3 < 0 if bright_surface else l3 > 0

        RB = l2.abs() / (l3.abs() + eps)
        S = torch.sqrt(l1**2 + l2**2 + l3**2)
        gamma_scale = gamma if gamma is not None else 0.5 * S.max()

        term1 = torch.exp(-(RB**2) / (2 * beta**2))
        term2 = 1 - torch.exp(-(S**2) / (2 * gamma_scale**2 + eps))

        response = torch.zeros_like(volume)
        response[valid] = (term1 * term2)[valid]

        output = torch.maximum(output, response)

    # Normalize
    output = output / (output.max() + eps)
    return output.squeeze()


In [ ]:
logits_pred = logits_pred.squeeze()
print(f"logits_pred: {logits_pred.shape}")
#prob = torch.sigmoid(logits_pred)[0][0].to("cuda")  # shape (D, H, W) or (1, D, H, W)
#surf = frangi_surfaceness_3d_torch(prob, sigmas=(0.5, 1.0, 1.5, 2.0))

# Baseline
prob = torch.sigmoid(logits_pred).to("cuda")

CROP_MARGIN = 9
prob[:CROP_MARGIN, :, :] = 0
prob[-CROP_MARGIN:, :, :] = 0
prob[:, :CROP_MARGIN, :] = 0
prob[:, -CROP_MARGIN:, :] = 0
prob[:, :, :CROP_MARGIN] = 0
prob[:, :, -CROP_MARGIN:] = 0

#binary_mask = (prob > 0.75).float()  # threshold similar to baseline
#surf = frangi_surfaceness_3d_torch(binary_mask, sigmas=(1.0,1.5,2.0))
surf = frangi_surfaceness_3d_torch(prob, sigmas=(0.8, 1.2, 1.6, 2.0))  # slightly larger sigma
enhanced = prob + 0.4 * surf * (1 - prob)
binary = enhanced > 0.5
binary_original = np.transpose(binary.squeeze().cpu().numpy(), (2, 1, 0))



logits_pred: torch.Size([320, 320, 320])


In [ ]:
"""# fuse safely
enhanced = binary_mask + 0.5 * surf  # alpha=0.5, baseline-style
binary = enhanced > 0.5          # threshold after fusion
binary = np.transpose(binary.squeeze().cpu().numpy(), (2, 1, 0))"""

In [ ]:
#enhanced = prob + 0.4 * surf * (1 - prob)
#binary = enhanced > 0.75
#binary = np.transpose(binary.squeeze().cpu().numpy(), (2, 1, 0))

In [ ]:
#enhanced = prob_pred * (1 + surf)
#binary = enhanced > 0.75
#binary = np.transpose(binary.squeeze().cpu().numpy(), (2, 1, 0))
#binary = surf > 0.2
#binary = np.transpose(binary.squeeze().cpu().numpy(), (2, 1, 0))

In [32]:
from scipy.ndimage import binary_opening
import numpy as np

def erode_2d_yz(vol, s):
    """
    Apply 2D binary opening on each slice along the yz-plane
    to remove bridges along x while preserving sheets in y-z.
    
    Parameters
    ----------
    vol : ndarray (z, y, x)
    s : int : structuring element size along y and z
    """
    z, y, x = vol.shape
    se2 = np.ones((s, s))
    
    # Apply opening slice by slice along x (yz-plane)
    vol_yz = np.stack([binary_opening(vol[:, :, i], structure=se2) for i in range(x)], axis=2)
    return vol_yz

binary = erode_2d_yz(vol=binary, s=3)

In [71]:
import numpy as np
from scipy import ndimage as ndi

def competitive_hysteresis(probs, low=0.55, high=0.85):
    """
    Grows high-confidence sheets into low-confidence areas,
    but prevents them from merging if they were originally separate.
    """
    # 1. Create Masks
    mask_high = probs > high  # The "Seeds" (Separated sheets)
    mask_low  = probs > low   # The "Land" (Full shape with bridges)

    # 2. Label the High Confidence Seeds
    # Each sheet gets a unique ID (1, 2, 3...)
    seeds, num_seeds = ndi.label(mask_high)
    
    # 3. Geodesic Reconstruction (Dilation)
    # We want to reconstruct 'seeds' inside 'mask_low'.
    # Standard 'reconstruction' would merge them if the low-mask connects them.
    # To prevent merging, we can use a Watershed on the distance transform 
    # constrained to the 'mask_low'.
    
    # Calculate distance from the seeds *within* the low mask
    # We invert logic: distance typically grows from background, so we construct
    # a cost map where high-prob areas are 'cheap' to travel through.
    cost_map = -probs 
    
    # Use Watershed
    # seeds: The markers (1, 2, 3...)
    # mask: The valid area to grow into (mask_low)
    from skimage.segmentation import watershed
    
    reconstructed_labels = watershed(cost_map, seeds, mask=mask_low)
    
    return reconstructed_labels > 0

# Usage
binary_original = competitive_hysteresis(enhanced.squeeze().cpu().numpy(), low=0.75, high=0.90)

# If bigger than a threshold of voxels, destroy harder individual elements (for cases with bridges)

In [ ]:
import numpy as np

# Find the sheets bigger than threshold = 50000
THRESHOLD = 50000

# Remove small objects
binary_original = split_by_object_np(binary_original)
binary_original = remove_small_objects(binary_original, min_size=5000, connectivity=1)
binary_original[binary_original>0.5] = 1
categorical_mask = split_by_object_np(binary_original) 

# Kepp only the volumes bigger than THRESHOLD
labels, counts = np.unique(categorical_mask, return_counts=True)
keep_labels = labels[(counts > THRESHOLD) & (labels != 0)]
filtered_mask = np.where(np.isin(categorical_mask, keep_labels), categorical_mask, 0)

# Make the filtered mask binary
filtered_mask[filtered_mask>0.5] = 1

# Remove the first prediction
binary_original[filtered_mask==1] = 0

# Add now the more certain volume in the region removed before
binary_certain = enhanced > 0.90
binary_certain = np.transpose(binary_certain.squeeze().cpu().numpy(), (2, 1, 0))
binary_original[binary_certain==1] = 1
binary_original = split_by_object_np(binary_original)
binary_original = remove_small_objects(binary_original, min_size=5000, connectivity=1)
binary_original[binary_original>0.5] = 1

#binary_original = apply_hysteresis_threshold(binary_original, 0.3, 0.75)

output_path_final = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/binary_original_certain_1013184726.tif"
tifffile.imwrite(output_path_final, binary_original.astype(np.uint16))

_ = split_by_object(
    output_path_final, 
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_certain_1013184726.tif")

print(f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_certain_1013184726.nii.gz")

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_certain_1013184726.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_certain_1013184726.nii.gz


In [64]:
import numpy as np
from scipy import ndimage as ndi

import numpy as np
from scipy import ndimage as ndi

def close_tunnels_per_label_fast(volume, hole_radius=2):
    """
    Optimized version: Process bounding boxes instead of full volumes.
    """
    # 1. Label the volume (if not already labeled)
    # Ensure this returns integer labels. 
    # If 'volume' is already labeled (int), you can skip split_by_object_np.
    if volume.dtype == bool or (np.issubdtype(volume.dtype, np.integer) and volume.max() <= 1):
        # Use existing function or fall back to ndi.label
        try:
            labeled_volume = split_by_object_np(volume)
        except NameError:
            labeled_volume, _ = ndi.label(volume)
    else:
        labeled_volume = volume

    # 2. Find Bounding Boxes for all objects at once
    # This is extremely fast and returns a list of slice tuples 
    # e.g. [(slice(0,10), slice(5,15), ...), ...]
    object_slices = ndi.find_objects(labeled_volume)
    
    # 3. Define Footprint (Structuring Element)
    structure_size = 2 * hole_radius + 1
    z, y, x = np.ogrid[:structure_size, :structure_size, :structure_size]
    center = hole_radius
    footprint = ((z-center)**2 + (y-center)**2 + (x-center)**2 <= hole_radius**2)

    # 4. Process each object in its own little box
    output_volume = np.zeros_like(labeled_volume)

    # Note: ndi.find_objects returns slices for labels 1, 2, 3... in order.
    for idx, slices in tqdm(enumerate(object_slices), total=len(object_slices), desc="Closing tunnels"):
        if slices is None: continue # Label index might be missing in sequence
        
        label_id = idx + 1
        
        # --- KEY OPTIMIZATION: Expand the slice (Padding) ---
        # We must pad the bounding box by 'hole_radius'. 
        # If we don't, the 'closing' (dilation) will hit the edge of the crop 
        # and be cut off, creating artifacts.
        padded_slices = []
        for dim, s in enumerate(slices):
            # Pad start and stop, clipping to volume boundaries
            start = max(0, s.start - hole_radius)
            stop = min(labeled_volume.shape[dim], s.stop + hole_radius)
            padded_slices.append(slice(start, stop))
        padded_slices = tuple(padded_slices)

        # --- Extract Crop ---
        # This creates a view or small copy, much smaller than full volume
        sub_vol = labeled_volume[padded_slices]
        
        # --- Local Mask ---
        binary_sheet = (sub_vol == label_id)
        
        # --- Apply Closing (Fast on small array) ---
        closed_sheet = ndi.binary_closing(binary_sheet, structure=footprint)
        
        # --- Write Back ---
        # We only update the specific region in the output volume
        # Use boolean indexing on the crop to assign the label
        output_crop = output_volume[padded_slices]
        output_crop[closed_sheet] = label_id

    return output_volume

binary_original = close_tunnels_per_label_fast(binary_original, hole_radius=9)

Closing tunnels: 100%|██████████| 27/27 [05:57<00:00, 13.26s/it]


In [39]:
from tqdm import tqdm
from skimage.measure import regionprops
import cupy as cp
import cupyx.scipy.ndimage as ndimage_gpu
from skimage import morphology

labeled_volume = split_by_object_np(binary_original) 
labeled_volume = remove_small_objects(labeled_volume, min_size=1000, connectivity=1)
labels = split_by_object_np(labeled_volume) 

DILATION_RADIUS = 9
EROSION_RADIUS = 9
MEDIAN_FILTER_SIZE = 1

print("Creating kernels and moving to GPU...")
# 1. Create balls on CPU
cpu_ball_dilate = morphology.ball(DILATION_RADIUS)
cpu_ball_erode = morphology.ball(EROSION_RADIUS)

# 2. Move to GPU (do this ONCE, not in the loop)
gpu_ball_dilate = cp.array(cpu_ball_dilate)
gpu_ball_erode = cp.array(cpu_ball_erode)


PAD_WIDTH = 3
dtype_out =  np.uint16
filled_labels = np.zeros_like(labels, dtype=dtype_out)
for region in tqdm(regionprops(labeled_volume), desc=f"Processing"):

    label_id = region.label

    # --- CPU PART (Slicing is fast on CPU) ---
    bbox = region.bbox
    minr, minc, mins, maxr, maxc, maxs = bbox
    slices = (slice(minr, maxr), slice(minc, maxc), slice(mins, maxs))

    # Extract binary sub-volume
    sub = (labeled_volume[slices] == label_id)

    # 2. Pad (CPU)
    sub_padded = np.pad(sub, PAD_WIDTH, mode='constant', constant_values=0)

    # --- GPU PART (Heavy lifting) ---
    # 3. Transfer to VRAM
    gpu_sub = cp.array(sub_padded)

    # 4. Solidify (Dilation + Erosion)
    # Note: We use 'structure=' to pass our pre-loaded ball
    gpu_sub = ndimage_gpu.binary_dilation(gpu_sub, structure=gpu_ball_dilate)
    gpu_sub = ndimage_gpu.binary_erosion(gpu_sub, structure=gpu_ball_erode)

    # 5. Fill Internal Holes
    gpu_sub = ndimage_gpu.binary_fill_holes(gpu_sub)

    # 6. Ironing / Straightening (Median Filter)
    if MEDIAN_FILTER_SIZE > 0:
        gpu_sub = ndimage_gpu.median_filter(gpu_sub, size=MEDIAN_FILTER_SIZE)

    # --- RETRIEVE PART ---
    # 7. Pull back to CPU
    sub_filled = gpu_sub.get()

    # 8. Un-pad
    sub_final = sub_filled[PAD_WIDTH:-PAD_WIDTH, PAD_WIDTH:-PAD_WIDTH, PAD_WIDTH:-PAD_WIDTH]

    # 9. Write back
    out_block = filled_labels[slices]
    out_block[sub_final] = label_id
    filled_labels[slices] = out_block
    
binary_original = filled_labels

Creating kernels and moving to GPU...


Processing: 100%|██████████| 29/29 [00:14<00:00,  2.04it/s]


In [72]:
output_path_final = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/frangi_surfaceness_3d_1013184726.tif"
tifffile.imwrite(output_path_final, binary_original.astype(np.uint16))
print(f"Saved closed volume to: {output_path_final}")

_ = split_by_object(
    output_path_final, 
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_holes_1013184726.tif")

gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/1013184726.tif"
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=output_path_final,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report

Saved closed volume to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/frangi_surfaceness_3d_1013184726.tif
Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_3/categorical_rm_holes_1013184726.tif
Done!


{'image_score': np.float64(0.2668996050404696),
 'topo_score': 0.008774570949288142,
 'surface_dice': 0.2724496970251618,
 'voi_score': 0.48259954227679,
 'voi_split': 2.2499084304029036,
 'voi_merge': 1.3237961451839801}

In [ ]:
# Os resultados são muito afetados pelas pontes e pelos buracos
# Quando há pontes, o DSc desce porque ao encher os burcaros cria mais pontes entre eles, criando muito más segmentações. 
# Tentar criar o mínimo de pontes possiveis e depois encher os buracos parece a opção melhor
{'image_score': np.float64(0.47798227345567024),
 'topo_score': 0.12032620403138944,
 'surface_dice': 0.7196147723640545,
 'voi_score': 0.5429121197680981,
 'voi_split': 1.6856426999634104,
 'voi_merge': 1.1207530535343784}

# baseline
{'image_score': np.float64(0.48135781602218675),
 'topo_score': 0.11849382528979469,
 'surface_dice': 0.7291011117407524,
 'voi_score': 0.5446407980742428,
 'voi_split': 1.6841425705660977,
 'voi_merge': 1.102765813429556}

In [ ]:
## 2536049117
{'image_score': np.float64(0.7855649923931747),
 'topo_score': 0.8148148148148148,
 'surface_dice': 0.9261704128848708,
 'voi_score': 0.6198882955400724,
 'voi_split': 1.0055659495696496,
 'voi_merge': 1.0384136254318896}

# Compare two results

In [33]:
origin_pred = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/BCE_Tversky_DAsafe_epoch_model_best_TTA_overlap_0.5_th_0.5.csv"
post_process = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/results_post.csv"
df1 = pd.read_csv(origin_pred)
df2 = pd.read_csv(post_process)

df1['id'] = df1['id'].astype(str)
df2['id'] = df2['id'].astype(str)

# 2. Merge
merged_df = pd.merge(df1, df2, on='id', suffixes=('_1', '_2'))

# 3. Calculate difference
merged_df['difference'] = (merged_df['surface_dice_1'] - merged_df['surface_dice_2']).abs()

# 4. Sort and view
result = merged_df.sort_values(by='difference', ascending=False)
result


,tif_paths_1,id,pred_paths_1,image_score_1,topo_score_1,surface_dice_1,voi_score_1,voi_split_1,voi_merge_1,tif_paths_2,pred_paths_2,image_score_2,topo_score_2,surface_dice_2,voi_score_2,voi_split_2,voi_merge_2,difference
70,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3545147380,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.530518,0.079482,0.898432,0.549207,1.132644,1.603384,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.511396,0.312285,0.640790,0.552668,1.256705,1.441316,0.257642
49,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1796762532,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.455817,0.128292,0.716930,0.475441,1.320494,2.357207,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.350261,0.032163,0.472553,0.500625,1.381204,1.943801,0.244377
11,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1734818300,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.410514,0.085487,0.654723,0.444899,1.868641,2.290368,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.380814,0.211443,0.451983,0.454821,1.701104,2.294460,0.202740
8,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3897872090,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.605422,0.261549,0.945433,0.560159,0.720299,1.897058,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.605860,0.476190,0.743014,0.579853,0.644431,1.770819,0.202419
52,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,928131323,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.609732,0.265672,0.936340,0.578034,1.162442,1.270903,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.561472,0.352724,0.734871,0.566999,0.988218,1.557350,0.201468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,114235076,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.363203,0.009510,0.395123,0.634450,1.124610,0.795953,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.350638,0.020299,0.352294,0.632130,1.102530,0.837315,0.042829
62,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1013184726,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.488282,0.122576,0.740974,0.549053,1.675571,1.062153,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.519370,0.271600,0.700940,0.550175,1.638450,1.086890,0.040034
25,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,821686014,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.444000,0.055543,0.600934,0.620030,0.876180,1.166573,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.428734,0.046190,0.562462,0.622901,0.839672,1.178298,0.038472
75,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2536049117,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.624496,0.198421,0.952325,0.661873,0.805978,0.896902,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.773247,0.750000,0.915851,0.650569,0.893910,0.896478,0.036474


# Vesuvius post-processing proposal

In [128]:
import sys
sys.path.append("../utils")
from infer_class import VesuviusInferer
import os
import json
import sys
import os
sys.path.append(os.path.abspath("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/topological-metrics-kaggle/src"))
from torch.nn.functional import sigmoid
import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence
import sys
import os
import tifffile

case_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images/3545147380.tif"

config_content = {
  "architecture": "STU-Net",
  "patch_size": [128, 128, 128],
  "device": "cuda",
  "batch_size": 1,
  "num_workers": 4,

  "infer_sw_batch_size": 1,
  "blend_mode": "gaussian",
  "deep_supervision": False,
  "activation": False,

  "infer_overlap": 0.5,
  "TTA": True,
  "TH": 0.5,

  "checkpoint_path": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth", 
  "dataset_path_imgs": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images",
  "pred_save_dir": "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash"
}


# 🧪 Initialize Inference Object
infer_object = VesuviusInferer(config_content)

print(f"🚀 Starting inference on dataset: {config_content['dataset_path_imgs']}")
print("📋 Using the following configuration:")
for key, value in config_content.items():
    print(f"  🔹 {key}: {value}")


### Select manually a case to run!
input_data = {
    'image': str(case_path),
    'gt': None
}

logits_pred, pred_list = infer_object.infer(
    input=input_data, 
    test=True, 
    threshold_list=[0.5] # CHANGED FROM 0.5
)
infer_object.save_nifti(
    torch_tensor=pred_list[0], 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/3545147380.nii.gz", 
    real_file=None
)

infer_object.save_tiff(
    torch_tensor=pred_list[0], 
    save_path="/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/3545147380.tif"
)

Loading weights from: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth
🚀 Starting inference on dataset: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
📋 Using the following configuration:
  🔹 architecture: STU-Net
  🔹 patch_size: [128, 128, 128]
  🔹 device: cuda
  🔹 batch_size: 1
  🔹 num_workers: 4
  🔹 infer_sw_batch_size: 1
  🔹 blend_mode: gaussian
  🔹 deep_supervision: False
  🔹 activation: False
  🔹 infer_overlap: 0.5
  🔹 TTA: True
  🔹 TH: 0.5
  🔹 checkpoint_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/checkpoints/second_step/BCE_Tversky_DAsafe/model_best.pth
  🔹 dataset_path_imgs: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images
  🔹 pred_save_dir: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash
Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


  0%|          | 0/27 [00:00<?, ?it/s]Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
100%|██████████| 27/27 [00:17<00:00,  1.57it/s]


Doing axes: (3,)


100%|██████████| 27/27 [00:17<00:00,  1.56it/s]


Doing axes: (4,)


100%|██████████| 27/27 [00:17<00:00,  1.55it/s]


Doing axes: (3, 4)


100%|██████████| 27/27 [00:17<00:00,  1.55it/s]


In [100]:
numpy_array = tifffile.imread(case_path)
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/vol_2536049117.nii.gz"
nii_data = np.transpose(numpy_array, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)


numpy_array = tifffile.imread("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/2536049117.tif")
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/gt_2536049117.nii.gz"
nii_data = np.transpose(numpy_array, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

In [102]:
# No post-process
prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
prob_pred = (prob_pred > 0.5).astype(np.uint16)
out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/no_post_pred_2536049117.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/2536049117.tif"
tifffile.imwrite(out_path, prob_pred)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_no_post_pred_2536049117.tif")
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_no_post_pred_2536049117.tif
Done!


{'image_score': np.float64(0.6075685870946677),
 'topo_score': 0.14337035751475186,
 'surface_dice': 0.951139313542954,
 'voi_score': 0.6618820574291663,
 'voi_split': 0.8086779664586242,
 'voi_merge': 0.8941326716588209}

In [109]:
from skimage.filters import apply_hysteresis_threshold

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
#prob_pred = (prob_pred > 0.8).astype(np.uint16)
prob_pred = apply_hysteresis_threshold(prob_pred, 0.5, 0.50)
out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/hysteresis_th0375_mask_3545147380.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif"
tifffile.imwrite(out_path, prob_pred)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_3545147380.tif")

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
print(pred_path)
score_report

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_3545147380.tif
Done!
/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_Tversky_DAsafe_epoch_model_best/TTA/overlap_0.5/th_0.5/2536049117.tif


{'image_score': np.float64(0.5510487366220043),
 'topo_score': 0.0067346938775510205,
 'surface_dice': 0.8299907536265267,
 'voi_score': 0.7386616133984414,
 'voi_split': 0.22290054681191313,
 'voi_merge': 0.9564323703687093}

In [103]:
def split_by_object_np(binary_mask):
    # Ensure the volume is treated as a binary mask for labeling
    # (This assumes all objects you want to separate are non-zero)
    binary_mask = binary_mask > 0

    # 2. Perform Connected Component Labeling
    # connectivity=None defaults to allowing diagonal connections. 
    # Use connectivity=1 for strict face-to-face connections only.
    labeled_volume = label(binary_mask, connectivity=None)

    # 3. Optimize Data Type to save space
    # We check how many objects were found to choose the smallest file size possible.
    num_features = np.max(labeled_volume)
    if num_features < 255:
        final_volume = labeled_volume.astype(np.uint8)
    elif num_features < 65535:
        final_volume = labeled_volume.astype(np.uint16)
    else:
        final_volume = labeled_volume.astype(np.uint32)
    return final_volume

In [129]:
# TODO don't use this code, it is the best until now and I don't want to mess with it. I will just run it on the other cases and submit to see if it generalizes well.
import numpy as np
from scipy import ndimage as ndi
from skimage.filters import apply_hysteresis_threshold
def crop_prob(prob, margin=9):
    """Crop probability map by margin."""
    prob[:margin, :, :] = 0
    prob[-margin:, :, :] = 0
    prob[:, :margin, :] = 0
    prob[:, -margin:, :] = 0
    prob[:, :, :margin] = 0
    prob[:, :, -margin:] = 0
    return prob

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))

prob_pred = crop_prob(prob_pred, margin=5)

# Remove small objects
import numpy as np
from scipy import ndimage as ndi

def clean_and_split_seeds(seeds, min_size=1000):
    """
    Remove small seeds and split remaining seeds into separate objects.

    Parameters
    ----------
    seeds : np.ndarray
        Boolean array of seed voxels (True = seed)
    min_size : int
        Minimum voxel count to keep a seed

    Returns
    -------
    seeds_clean : np.ndarray
        Boolean array of cleaned seeds
    """
    # Label connected components
    labeled, num = ndi.label(seeds)

    # Count voxel sizes per component
    sizes = np.bincount(labeled.ravel())
    sizes[0] = 0  # ignore background

    # Keep only components >= min_size
    keep_labels = np.where(sizes >= min_size)[0]
    seeds_clean = np.isin(labeled, keep_labels)

    # Split each object individually
    seeds_clean = split_by_object_np(seeds_clean)

    return seeds_clean


import numpy as np
from scipy import ndimage as ndi

import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
import numpy as np
from scipy import ndimage as ndi

def grow_seeds_competitive(seeds_clean, prob_pred,
                           grow_threshold=0.5,
                           connectivity=1):
    """
    Competitive region growing (prevents sheet merging).

    Returns:
        final_labels (Z,Y,X) int32
    """

    # 1️⃣ Label seeds
    structure = ndi.generate_binary_structure(3, connectivity)
    seed_labels, _ = ndi.label(seeds_clean, structure=structure)

    # 2️⃣ Allowed growth region
    allowed = prob_pred > grow_threshold

    # 3️⃣ Use negative probability as "height"
    # Watershed expands from seeds inside allowed region
    elevation = -prob_pred

    final_labels = watershed(
        elevation,
        markers=seed_labels,
        mask=allowed,
        connectivity=structure
    )

    return final_labels.astype(np.int16)

def enforce_label_gap(labels, gap=3):
    """
    Remove only voxels that are within `gap` voxels
    of a different label (26-connectivity safe).
    """

    output = labels.copy()

    # 26-connectivity structure
    structure = ndi.generate_binary_structure(3, 3)

    # Dilate labels `gap` times
    dilated = np.zeros_like(labels, dtype=np.int16)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl
        expanded = ndi.binary_dilation(
            mask,
            structure=structure,
            iterations=gap
        )

        # mark expanded region with label id
        dilated[expanded & (dilated == 0)] = lbl
        # mark conflicts as -1
        conflict = expanded & (dilated != 0) & (dilated != lbl)
        dilated[conflict] = -1

    # Remove conflict voxels from original labels
    conflict_zone = dilated == -1
    output[conflict_zone] = 0

    return output.astype(np.int16)

def small_label_erosion(labels, iterations=1):
    """
    Perform a very small erosion per label.
    
    Parameters
    ----------
    labels : int array (Z,Y,X)
        Labeled volume (0=background, 1..N=labels)
    iterations : int
        Number of erosion iterations (default=1)

    Returns
    -------
    eroded_labels : int array
        Labels slightly shrunk
    """

    structure = ndi.generate_binary_structure(3, 3)  # 26-connectivity
    eroded_labels = np.zeros_like(labels)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl
        eroded = ndi.binary_erosion(mask, structure=structure, iterations=iterations)
        dilated = ndi.binary_dilation(eroded, structure=structure, iterations=iterations)
        eroded_labels[dilated] = lbl

    return eroded_labels.astype(np.int16)

import numpy as np
from scipy import ndimage as ndi
def pad_volume(volume, pad_size=32):
    """
    Pad a 3D volume with zeros on all sides.
    
    Args:
        volume (np.ndarray): 3D array (D, H, W)
        pad_size (int or tuple): number of voxels to pad on each side. 
                                 If int, same padding for all dimensions.
                                 If tuple of 3 ints, specifies (D, H, W).
    
    Returns:
        padded_volume (np.ndarray): zero-padded volume
        pad_width (tuple): padding applied ((before_D, after_D), ...)
    """
    if isinstance(pad_size, int):
        pad_size = (pad_size, pad_size, pad_size)
    
    pad_width = tuple((p, p) for p in pad_size)
    padded_volume = np.pad(volume, pad_width=pad_width, mode='constant', constant_values=0)
    
    return padded_volume, pad_width

def unpad_volume(padded_volume, pad_width):
    """
    Remove padding from a 3D volume.
    
    Args:
        padded_volume (np.ndarray): zero-padded 3D volume
        pad_width (tuple): padding applied ((before_D, after_D), ...)
    
    Returns:
        volume (np.ndarray): volume with padding removed
    """
    slices = tuple(slice(pw[0], -pw[1] if pw[1] > 0 else None) for pw in pad_width)
    return padded_volume[slices]

def solidify_sheets_by_label_force(labels, connectivity=3, iterations=3):
    labels, pad_w = pad_volume(labels, pad_size=iterations)

    solid_labels = np.zeros_like(labels, dtype=np.int32)
    structure = ndi.generate_binary_structure(3, connectivity)
    

    for lbl in np.unique(labels):
        if lbl == 0:
            continue
        
        
    
        mask = labels == lbl

        # Optional: first fill internal voids
        mask = ndi.binary_fill_holes(mask, structure=structure)

        # Then a *small* closing to aggressively seal thin tunnels
        dilated = ndi.binary_dilation(mask, structure=structure, iterations=iterations)
        closed = ndi.binary_erosion(dilated, structure=structure, iterations=iterations) 

        solid_labels[closed] = lbl
    solid_labels = unpad_volume(solid_labels, pad_w)
    return solid_labels

import numpy as np
import scipy.ndimage as ndi

def enforce_min_thickness(labels, target_min_thick=2.0, connectivity=3):
    thick_labels = np.zeros_like(labels, dtype=np.int32)
    half = target_min_thick / 2.0

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl            # sheet
        if not mask.any():
            continue

        # Distance from every background voxel to the sheet surface
        dist_to_sheet = ndi.distance_transform_edt(~mask)

        # Enforce that any background voxel closer than half the target is filled
        # This will grow the sheet just enough to eliminate < 3‑voxel thin regions
        enforced = mask | (dist_to_sheet < half)

        thick_labels[enforced] = lbl

    return thick_labels

#grown_seeds = grow_seeds_competitive(seeds_clean, prob_pred,
#                           grow_threshold=0.5,
#                           connectivity=1)



#grown_seeds = enforce_label_gap(grown_seeds)
#grown_seeds = apply_hysteresis_threshold(prob_pred, 0.45, 0.75)
grown_seeds = prob_pred>0.5
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = solidify_sheets_by_label_force(grown_seeds, connectivity=3, iterations=1)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
"""grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = small_label_erosion(grown_seeds, iterations=1)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = enforce_label_gap(grown_seeds)


#grown_seeds = enforce_label_gap(grown_seeds)
#grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
#grown_seeds = enforce_min_thickness(grown_seeds, target_min_thick=3.0, connectivity=3)
#grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = enforce_label_gap(grown_seeds)"""


out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/hysteresis_th0375_mask_2536049117.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif"
tifffile.imwrite(out_path, grown_seeds)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_2536049117.tif")
score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report



Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_2536049117.tif
Done!


{'image_score': np.float64(0.5398912721265068),
 'topo_score': 0.11573710395391464,
 'surface_dice': 0.8952125287867327,
 'voi_score': 0.548130731042788,
 'voi_split': 1.1320521080436656,
 'voi_merge': 1.6158888688443123}

In [ ]:
import os
import numpy as np
import tifffile
from tqdm import tqdm
from scipy import ndimage as ndi

# --- Configuration ---
INPUT_DIR = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images"
OUTPUT_DIR = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated_ndi_binary_propagation_45_75"
GT_DIR = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Core Functions ---

def crop_prob(prob, margin=5):
    """Crop probability map by margin."""
    prob[:margin, :, :] = 0
    prob[-margin:, :, :] = 0
    prob[:, :margin, :] = 0
    prob[:, -margin:, :] = 0
    prob[:, :, :margin] = 0
    prob[:, :, -margin:] = 0
    return prob

def clean_and_split_seeds(seeds, min_size=1000):
    """Label components and remove those smaller than min_size."""
    labeled, _ = ndi.label(seeds)
    sizes = np.bincount(labeled.ravel())
    sizes[0] = 0  
    keep_labels = np.where(sizes >= min_size)[0]
    seeds_clean = np.isin(labeled, keep_labels)
    
    # Assuming split_by_object_np is available in your environment
    return split_by_object_np(seeds_clean)

def pad_volume(volume, pad_size=32):
    if isinstance(pad_size, int):
        pad_size = (pad_size, pad_size, pad_size)
    pad_width = tuple((p, p) for p in pad_size)
    padded_volume = np.pad(volume, pad_width=pad_width, mode='constant', constant_values=0)
    return padded_volume, pad_width

def unpad_volume(padded_volume, pad_width):
    slices = tuple(slice(pw[0], -pw[1] if pw[1] > 0 else None) for pw in pad_width)
    return padded_volume[slices]

def solidify_sheets_by_label_force(labels, connectivity=3, iterations=1):
    """Fill holes and seal thin tunnels via closing (dilation then erosion)."""
    labels, pad_w = pad_volume(labels, pad_size=iterations)
    solid_labels = np.zeros_like(labels, dtype=np.int32)
    structure = ndi.generate_binary_structure(3, connectivity)

    for lbl in np.unique(labels):
        if lbl == 0: continue
        
        mask = labels == lbl
        mask = ndi.binary_fill_holes(mask, structure=structure)
        
        dilated = ndi.binary_dilation(mask, structure=structure, iterations=iterations)
        closed = ndi.binary_erosion(dilated, structure=structure, iterations=iterations) 
        solid_labels[closed] = lbl
        
    return unpad_volume(solid_labels, pad_w)

# --- Processing Loop ---

tif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.tif')]

for filename in tqdm(tif_files, desc="Running Inference & Post-Process"):
    file_id = filename.replace(".tif", "")
    out_path = os.path.join(OUTPUT_DIR, f"{filename}")
    if os.path.exists(out_path):
        print(f"Output already exists for {filename}, skipping.")
        continue
    # 1. Inference 
    # (Ensure input_data for the specific file is loaded here)
    input_data = os.path.join(INPUT_DIR, filename)
    print(f"Doing: {input_data}")
    logits_pred, _ = infer_object.infer(
        input={'image': str(input_data), 'gt': None}, 
        test=True, 
        threshold_list=[0.5])

    # 2. Probability Prep
    prob_pred = sigmoid(logits_pred)
    prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))
    prob_pred = crop_prob(prob_pred, margin=5)

    # 3. Refined Logic (Threshold -> Clean -> Solidify -> Clean)
    #grown_seeds = prob_pred > 0.5 # first try doing th=0.5
    grown_seeds = prob_pred > 0.45
    strong_seeds = prob_pred > 0.75
    grown_seeds = ndi.binary_propagation(strong_seeds, mask=grown_seeds) # Second try
    grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
    grown_seeds = solidify_sheets_by_label_force(grown_seeds, connectivity=3, iterations=1)
    grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)

    grown_seeds = grown_seeds>0.5

    # 4. Save
    
    tifffile.imwrite(out_path, grown_seeds.astype(np.uint16))



Running Inference & Post-Process:   0%|          | 0/79 [00:00<?, ?it/s]

Doing: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images/1669670049.tif
Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)
100%|██████████| 27/27 [00:17<00:00,  1.55it/s]


Doing axes: (3,)


100%|██████████| 27/27 [00:16<00:00,  1.59it/s]


...

Doing: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_images/4024884955.tif
Doing inference to these axes combinations:
[(), (3,), (4,), (3, 4)]
Doing axes: ()


100%|██████████| 64/64 [00:42<00:00,  1.52it/s]


Doing axes: (3,)


100%|██████████| 64/64 [00:42<00:00,  1.51it/s]


Doing axes: (4,)


In [ ]:
import sys
sys.path.append("../utils")
from infer_class import VesuviusInferer
from compute_metrics_obj import VesuviusMetric
def create_dfs(path_dir):
    # Generate DataFrame
    result_df = glob.glob(os.path.join(path_dir, '**/*.tif'), recursive=True)
    result_df = pd.DataFrame({'tif_paths': result_df})
    result_df['id'] = result_df['tif_paths'].apply(lambda x: os.path.basename(x).split('.')[0])

    # save dataframe to csv (take name from the path_dir last folder)
    csv_name = os.path.basename(os.path.normpath(path_dir)) + '_df.csv'
    result_df.to_csv(os.path.join(path_dir, csv_name), index=False)
    return result_df

create_dfs("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated_ndi_binary_propagation_45_75/")

test_metric_obj = VesuviusMetric(
    solution_path=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/val_labels_df.csv",
    submission_path=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated_ndi_binary_propagation_45_75/post_dilated_ndi_binary_propagation_45_75_df.csv",
    output_file=f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated_ndi_binary_propagation_45_75/results_post.csv"
)

test_metric_obj._run()

  0%|          | 0/79 [00:00<?, ?it/s]

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/1669670049.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/1669670049.tif


  3%|▎         | 2/79 [00:20<12:51, 10.02s/it]

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/541673255.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/541673255.tif


  4%|▍         | 3/79 [00:51<23:46, 18.76s/it]

...

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/149409101.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/149409101.tif


100%|██████████| 79/79 [41:56<00:00, 28.30s/it]

gt_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3686818985.tif
pred_path: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/3686818985.tif


100%|██████████| 79/79 [42:25<00:00, 32.22s/it]


📊 FINAL AGGREGATED METRICS
  id  image_score  topo_score  surface_dice  voi_score  voi_split  voi_merge
MEAN     0.566194    0.278841      0.829088   0.549603   1.340478   1.427932
✅ Saved detailed metrics with Mean row to: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_dilated/results_post.csv


In [ ]:
# grow_seeds_competitive(seeds_clean, prob_pred,grow_threshold=0.4, connectivity=1)
{'image_score': np.float64(0.5476696311406051),
 'topo_score': 0.15965118229079323,
 'surface_dice': 0.8748016197645412,
 'voi_score': 0.5531248843879362,
 'voi_split': 1.3989625523419127,
 'voi_merge': 1.2940707223320127}

# Added grown_seeds = ndi.binary_dilation(grown_seeds, structure=ndi.generate_binary_structure(3, 1), iterations=1)
{'image_score': np.float64(0.5803267385474029),
 'topo_score': 0.2799794132784354,
 'surface_dice': 0.867418939951312,
 'voi_score': 0.5506751016597516,
 'voi_split': 1.3689204956354908,
 'voi_merge': 1.350922214920875}

# When not using enforce_min_thickness
{'image_score': np.float64(0.6032754221973837),
 'topo_score': 0.38663823738450603,
 'surface_dice': 0.849665504849308,
 'voi_score': 0.5425743550993544,
 'voi_split': 1.0817019874739124,
 'voi_merge': 1.7285158846730508}

# having all
{'image_score': np.float64(0.5710288547765316),
 'topo_score': 0.24604251469923114,
 'surface_dice': 0.8678840045981637,
 'voi_score': 0.552733425021157,
 'voi_split': 1.3954791739179504,
 'voi_merge': 1.301822121377777}

In [ ]:
# TODO don't use this code, it is the best until now and I don't want to mess with it. I will just run it on the other cases and submit to see if it generalizes well.
# TODO second try: best one ever now
import numpy as np
from scipy import ndimage as ndi
def crop_prob(prob, margin=9):
    """Crop probability map by margin."""
    prob[:margin, :, :] = 0
    prob[-margin:, :, :] = 0
    prob[:, :margin, :] = 0
    prob[:, -margin:, :] = 0
    prob[:, :, :margin] = 0
    prob[:, :, -margin:] = 0
    return prob

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))

prob_pred = crop_prob(prob_pred, margin=3)

seeds = prob_pred>0.8

# Remove small objects
import numpy as np
from scipy import ndimage as ndi

def clean_and_split_seeds(seeds, min_size=1000):
    """
    Remove small seeds and split remaining seeds into separate objects.

    Parameters
    ----------
    seeds : np.ndarray
        Boolean array of seed voxels (True = seed)
    min_size : int
        Minimum voxel count to keep a seed

    Returns
    -------
    seeds_clean : np.ndarray
        Boolean array of cleaned seeds
    """
    # Label connected components
    labeled, num = ndi.label(seeds)

    # Count voxel sizes per component
    sizes = np.bincount(labeled.ravel())
    sizes[0] = 0  # ignore background

    # Keep only components >= min_size
    keep_labels = np.where(sizes >= min_size)[0]
    seeds_clean = np.isin(labeled, keep_labels)

    # Split each object individually
    seeds_clean = split_by_object_np(seeds_clean)

    return seeds_clean

seeds_clean = clean_and_split_seeds(seeds, min_size=1000)
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/seeds_clean.nii.gz"
nii_data = np.transpose(seeds_clean, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

import numpy as np
from scipy import ndimage as ndi

import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
import numpy as np
from scipy import ndimage as ndi

def grow_seeds_competitive(seeds_clean, prob_pred,
                           grow_threshold=0.5,
                           connectivity=1):
    """
    Competitive region growing (prevents sheet merging).

    Returns:
        final_labels (Z,Y,X) int32
    """

    # 1️⃣ Label seeds
    structure = ndi.generate_binary_structure(3, connectivity)
    seed_labels, _ = ndi.label(seeds_clean, structure=structure)

    # 2️⃣ Allowed growth region
    allowed = prob_pred > grow_threshold

    # 3️⃣ Use negative probability as "height"
    # Watershed expands from seeds inside allowed region
    elevation = -prob_pred

    final_labels = watershed(
        elevation,
        markers=seed_labels,
        mask=allowed,
        connectivity=structure
    )

    return final_labels.astype(np.int16)

def enforce_label_gap(labels, gap=3):
    """
    Remove only voxels that are within `gap` voxels
    of a different label (26-connectivity safe).
    """

    output = labels.copy()

    # 26-connectivity structure
    structure = ndi.generate_binary_structure(3, 3)

    # Dilate labels `gap` times
    dilated = np.zeros_like(labels, dtype=np.int16)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl
        expanded = ndi.binary_dilation(
            mask,
            structure=structure,
            iterations=gap
        )

        # mark expanded region with label id
        dilated[expanded & (dilated == 0)] = lbl
        # mark conflicts as -1
        conflict = expanded & (dilated != 0) & (dilated != lbl)
        dilated[conflict] = -1

    # Remove conflict voxels from original labels
    conflict_zone = dilated == -1
    output[conflict_zone] = 0

    return output.astype(np.int16)

def small_label_erosion(labels, iterations=1):
    """
    Perform a very small erosion per label.
    
    Parameters
    ----------
    labels : int array (Z,Y,X)
        Labeled volume (0=background, 1..N=labels)
    iterations : int
        Number of erosion iterations (default=1)

    Returns
    -------
    eroded_labels : int array
        Labels slightly shrunk
    """

    structure = ndi.generate_binary_structure(3, 3)  # 26-connectivity
    eroded_labels = np.zeros_like(labels)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl
        eroded = ndi.binary_erosion(mask, structure=structure, iterations=iterations)
        dilated = ndi.binary_dilation(eroded, structure=structure, iterations=iterations)
        eroded_labels[dilated] = lbl

    return eroded_labels.astype(np.int16)

import numpy as np
from scipy import ndimage as ndi
def pad_volume(volume, pad_size=32):
    """
    Pad a 3D volume with zeros on all sides.
    
    Args:
        volume (np.ndarray): 3D array (D, H, W)
        pad_size (int or tuple): number of voxels to pad on each side. 
                                 If int, same padding for all dimensions.
                                 If tuple of 3 ints, specifies (D, H, W).
    
    Returns:
        padded_volume (np.ndarray): zero-padded volume
        pad_width (tuple): padding applied ((before_D, after_D), ...)
    """
    if isinstance(pad_size, int):
        pad_size = (pad_size, pad_size, pad_size)
    
    pad_width = tuple((p, p) for p in pad_size)
    padded_volume = np.pad(volume, pad_width=pad_width, mode='constant', constant_values=0)
    
    return padded_volume, pad_width

def unpad_volume(padded_volume, pad_width):
    """
    Remove padding from a 3D volume.
    
    Args:
        padded_volume (np.ndarray): zero-padded 3D volume
        pad_width (tuple): padding applied ((before_D, after_D), ...)
    
    Returns:
        volume (np.ndarray): volume with padding removed
    """
    slices = tuple(slice(pw[0], -pw[1] if pw[1] > 0 else None) for pw in pad_width)
    return padded_volume[slices]

def solidify_sheets_by_label_force(labels, connectivity=3, iterations=3):
    labels, pad_w = pad_volume(labels, pad_size=iterations)

    solid_labels = np.zeros_like(labels, dtype=np.int32)
    structure = ndi.generate_binary_structure(3, connectivity)
    

    for lbl in np.unique(labels):
        if lbl == 0:
            continue
        
        
    
        mask = labels == lbl

        # Optional: first fill internal voids
        mask = ndi.binary_fill_holes(mask, structure=structure)

        # Then a *small* closing to aggressively seal thin tunnels
        dilated = ndi.binary_dilation(mask, structure=structure, iterations=iterations)
        closed = ndi.binary_erosion(dilated, structure=structure, iterations=iterations+1) 

        solid_labels[closed] = lbl
    solid_labels = unpad_volume(solid_labels, pad_w)
    return solid_labels

import numpy as np
import scipy.ndimage as ndi

def enforce_min_thickness(labels, target_min_thick=2.0, connectivity=3):
    thick_labels = np.zeros_like(labels, dtype=np.int32)
    half = target_min_thick / 2.0

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl            # sheet
        if not mask.any():
            continue

        # Distance from every background voxel to the sheet surface
        dist_to_sheet = ndi.distance_transform_edt(~mask)

        # Enforce that any background voxel closer than half the target is filled
        # This will grow the sheet just enough to eliminate < 3‑voxel thin regions
        enforced = mask | (dist_to_sheet < half)

        thick_labels[enforced] = lbl

    return thick_labels

grown_seeds = grow_seeds_competitive(seeds_clean, prob_pred,
                           grow_threshold=0.5,
                           connectivity=1)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = small_label_erosion(grown_seeds, iterations=1)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = enforce_label_gap(grown_seeds)


output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/grown_seeds_here.nii.gz"
nii_data = np.transpose(grown_seeds, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)



grown_seeds = solidify_sheets_by_label_force(grown_seeds, connectivity=3, iterations=11)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
#grown_seeds = enforce_min_thickness(grown_seeds, target_min_thick=3.0, connectivity=3)
#grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = ndi.binary_dilation(grown_seeds, structure=ndi.generate_binary_structure(3, 1), iterations=1)
grown_seeds = enforce_label_gap(grown_seeds)

output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/grown_seeds.nii.gz"
nii_data = np.transpose(grown_seeds, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/hysteresis_th0375_mask_3545147380.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif"
tifffile.imwrite(out_path, grown_seeds)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_3545147380.tif")

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report



In [68]:
# TODO don't use this code, it is the best until now and I don't want to mess with it. I will just run it on the other cases and submit to see if it generalizes well.
import numpy as np
from scipy import ndimage as ndi
def crop_prob(prob, margin=9):
    """Crop probability map by margin."""
    prob[:margin, :, :] = 0
    prob[-margin:, :, :] = 0
    prob[:, :margin, :] = 0
    prob[:, -margin:, :] = 0
    prob[:, :, :margin] = 0
    prob[:, :, -margin:] = 0
    return prob

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))

prob_pred = crop_prob(prob_pred, margin=9)

seeds = prob_pred>0.8

# Remove small objects
import numpy as np
from scipy import ndimage as ndi

def clean_and_split_seeds(seeds, min_size=1000):
    """
    Remove small seeds and split remaining seeds into separate objects.

    Parameters
    ----------
    seeds : np.ndarray
        Boolean array of seed voxels (True = seed)
    min_size : int
        Minimum voxel count to keep a seed

    Returns
    -------
    seeds_clean : np.ndarray
        Boolean array of cleaned seeds
    """
    # Label connected components
    labeled, num = ndi.label(seeds)

    # Count voxel sizes per component
    sizes = np.bincount(labeled.ravel())
    sizes[0] = 0  # ignore background

    # Keep only components >= min_size
    keep_labels = np.where(sizes >= min_size)[0]
    seeds_clean = np.isin(labeled, keep_labels)

    # Split each object individually
    seeds_clean = split_by_object_np(seeds_clean)

    return seeds_clean

seeds_clean = clean_and_split_seeds(seeds, min_size=1000)
output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/seeds_clean.nii.gz"
nii_data = np.transpose(seeds_clean, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

import numpy as np
from scipy import ndimage as ndi

import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
import numpy as np
from scipy import ndimage as ndi

def grow_seeds_competitive(seeds_clean, prob_pred,
                           grow_threshold=0.5,
                           connectivity=1):
    """
    Competitive region growing (prevents sheet merging).

    Returns:
        final_labels (Z,Y,X) int32
    """

    # 1️⃣ Label seeds
    structure = ndi.generate_binary_structure(3, connectivity)
    seed_labels, _ = ndi.label(seeds_clean, structure=structure)

    # 2️⃣ Allowed growth region
    allowed = prob_pred > grow_threshold

    # 3️⃣ Use negative probability as "height"
    # Watershed expands from seeds inside allowed region
    elevation = -prob_pred

    final_labels = watershed(
        elevation,
        markers=seed_labels,
        mask=allowed,
        connectivity=structure
    )

    return final_labels.astype(np.int16)

def enforce_label_gap(labels, gap=3):
    """
    Remove only voxels that are within `gap` voxels
    of a different label (26-connectivity safe).
    """

    output = labels.copy()

    # 26-connectivity structure
    structure = ndi.generate_binary_structure(3, 3)

    # Dilate labels `gap` times
    dilated = np.zeros_like(labels, dtype=np.int16)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl
        expanded = ndi.binary_dilation(
            mask,
            structure=structure,
            iterations=gap
        )

        # mark expanded region with label id
        dilated[expanded & (dilated == 0)] = lbl
        # mark conflicts as -1
        conflict = expanded & (dilated != 0) & (dilated != lbl)
        dilated[conflict] = -1

    # Remove conflict voxels from original labels
    conflict_zone = dilated == -1
    output[conflict_zone] = 0

    return output.astype(np.int16)

def small_label_erosion(labels, iterations=1):
    """
    Perform a very small erosion per label.
    
    Parameters
    ----------
    labels : int array (Z,Y,X)
        Labeled volume (0=background, 1..N=labels)
    iterations : int
        Number of erosion iterations (default=1)

    Returns
    -------
    eroded_labels : int array
        Labels slightly shrunk
    """

    structure = ndi.generate_binary_structure(3, 3)  # 26-connectivity
    eroded_labels = np.zeros_like(labels)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl
        eroded = ndi.binary_erosion(mask, structure=structure, iterations=iterations)
        dilated = ndi.binary_dilation(eroded, structure=structure, iterations=iterations)
        eroded_labels[dilated] = lbl

    return eroded_labels.astype(np.int16)

import numpy as np
from scipy import ndimage as ndi
def pad_volume(volume, pad_size=32):
    """
    Pad a 3D volume with zeros on all sides.
    
    Args:
        volume (np.ndarray): 3D array (D, H, W)
        pad_size (int or tuple): number of voxels to pad on each side. 
                                 If int, same padding for all dimensions.
                                 If tuple of 3 ints, specifies (D, H, W).
    
    Returns:
        padded_volume (np.ndarray): zero-padded volume
        pad_width (tuple): padding applied ((before_D, after_D), ...)
    """
    if isinstance(pad_size, int):
        pad_size = (pad_size, pad_size, pad_size)
    
    pad_width = tuple((p, p) for p in pad_size)
    padded_volume = np.pad(volume, pad_width=pad_width, mode='constant', constant_values=0)
    
    return padded_volume, pad_width

def unpad_volume(padded_volume, pad_width):
    """
    Remove padding from a 3D volume.
    
    Args:
        padded_volume (np.ndarray): zero-padded 3D volume
        pad_width (tuple): padding applied ((before_D, after_D), ...)
    
    Returns:
        volume (np.ndarray): volume with padding removed
    """
    slices = tuple(slice(pw[0], -pw[1] if pw[1] > 0 else None) for pw in pad_width)
    return padded_volume[slices]

def solidify_sheets_by_label_force(labels, connectivity=3, iterations=3):
    labels, pad_w = pad_volume(labels, pad_size=iterations)

    solid_labels = np.zeros_like(labels, dtype=np.int32)
    structure = ndi.generate_binary_structure(3, connectivity)

    for lbl in np.unique(labels):
        if lbl == 0:
            continue
        
        
    
        mask = labels == lbl

        # Optional: first fill internal voids
        mask = ndi.binary_fill_holes(mask, structure=structure)

        # Then a *small* closing to aggressively seal thin tunnels
        dilated = ndi.binary_dilation(mask, structure=structure, iterations=iterations)
        closed = ndi.binary_erosion(dilated, structure=structure, iterations=iterations+1) 

        solid_labels[closed] = lbl
    solid_labels = unpad_volume(solid_labels, pad_w)
    return solid_labels

import numpy as np
import scipy.ndimage as ndi

def enforce_min_thickness(labels, target_min_thick=2.0, connectivity=3):
    thick_labels = np.zeros_like(labels, dtype=np.int32)
    half = target_min_thick / 2.0

    for lbl in np.unique(labels):
        if lbl == 0:
            continue

        mask = labels == lbl            # sheet
        if not mask.any():
            continue

        # Distance from every background voxel to the sheet surface
        dist_to_sheet = ndi.distance_transform_edt(~mask)

        # Enforce that any background voxel closer than half the target is filled
        # This will grow the sheet just enough to eliminate < 3‑voxel thin regions
        enforced = mask | (dist_to_sheet < half)

        thick_labels[enforced] = lbl

    return thick_labels

grown_seeds = grow_seeds_competitive(seeds_clean, prob_pred,
                           grow_threshold=0.5,
                           connectivity=1)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = small_label_erosion(grown_seeds, iterations=1)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = enforce_label_gap(grown_seeds)


output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/grown_seeds_here.nii.gz"
nii_data = np.transpose(grown_seeds, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)



grown_seeds = solidify_sheets_by_label_force(grown_seeds, connectivity=3, iterations=11)
grown_seeds = enforce_label_gap(grown_seeds)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = enforce_min_thickness(grown_seeds, target_min_thick=3.0, connectivity=3)
grown_seeds = clean_and_split_seeds(grown_seeds, min_size=1000)
grown_seeds = enforce_label_gap(grown_seeds)

output_path_nii = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/grown_seeds.nii.gz"
nii_data = np.transpose(grown_seeds, (2, 1, 0)).astype(np.uint16)
nii_data = np.flip(nii_data, axis=0) 
nii_data = np.flip(nii_data, axis=1)
nib.save(nib.Nifti1Image(nii_data, np.eye(4)), output_path_nii)

out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/hysteresis_th0375_mask_3545147380.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif"
tifffile.imwrite(out_path, grown_seeds)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_3545147380.tif")

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report


Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_hysteresis_th0375_mask_3545147380.tif
Done!


{'image_score': np.float64(0.5710288547765316),
 'topo_score': 0.24604251469923114,
 'surface_dice': 0.8678840045981637,
 'voi_score': 0.552733425021157,
 'voi_split': 1.3954791739179504,
 'voi_merge': 1.301822121377777}

In [2]:
def build_anisotropic_struct(z_radius, xy_radius):
    z, r = z_radius, xy_radius
    if z == 0 and r == 0: return None
    depth = 2 * z + 1
    size = 2 * r + 1
    struct = np.zeros((depth, size, size), dtype=bool)
    cz, cy, cx = z, r, r
    for dz in range(-z, z + 1):
        for dy in range(-r, r + 1):
            for dx in range(-r, r + 1):
                if dy * dy + dx * dx <= r * r:
                    struct[cz + dz, cy + dy, cx + dx] = True
    return struct


In [3]:
def topo_postprocess(probs, T_low, T_high, z_radius, xy_radius, dust_min_size):
    # 1. Hysteresis
    strong = probs >= T_high
    weak = probs >= T_low
    
    if not strong.any(): return np.zeros_like(probs, dtype=np.uint8)
    
    struct_hyst = ndi.generate_binary_structure(3, 3)
    mask = ndi.binary_propagation(strong, mask=weak, structure=struct_hyst)
    
    if not mask.any(): return np.zeros_like(probs, dtype=np.uint8)

    # 2. Anisotropic Closing
    struct = build_anisotropic_struct(z_radius, xy_radius)
    if struct is not None:
        mask = ndi.binary_closing(mask, structure=struct)

    # 3. Dust Removal
    if dust_min_size > 0:
        mask = remove_small_objects(mask.astype(bool), min_size=dust_min_size)

    return mask.astype(np.uint8)


In [4]:
import numpy as np
import pandas as pd
import zipfile
import tifffile
import scipy.ndimage as ndi
from skimage.morphology import remove_small_objects
import gc

In [10]:
# Post-processing config (Tuned V24)
T_LOW = 0.30        # Keep connectivity permissive
T_HIGH = 0.80       # Lowered from 0.85 to find more seeds
Z_RADIUS = 0        # Increased from 1 to 2 (Fixes vertical splits)
XY_RADIUS = 0       # Keep 0 (Prevent mergers)
DUST_MIN_SIZE = 300 # Lowered from 500 to keep smaller valid fragments
CROP_MARGIN = 0

prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))

prob_pred[:CROP_MARGIN, :, :] = 0
prob_pred[-CROP_MARGIN:, :, :] = 0
prob_pred[:, :CROP_MARGIN, :] = 0
prob_pred[:, -CROP_MARGIN:, :] = 0
prob_pred[:, :, :CROP_MARGIN] = 0
prob_pred[:, :, -CROP_MARGIN:] = 0
final_mask = topo_postprocess(
            prob_pred, 
            T_low=T_LOW, 
            T_high=T_HIGH, 
            z_radius=Z_RADIUS, 
            xy_radius=XY_RADIUS, 
            dust_min_size=DUST_MIN_SIZE
        )


out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/final_pred_3545147380.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif"
tifffile.imwrite(out_path, final_mask)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_3545147380.tif")


score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)

score_report

# Original 0.5305181364493412	0.07948210753461608	0.8984319807505937	0.5492066026464245

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_3545147380.tif
Done!


{'image_score': np.float64(0.20629769104582085),
 'topo_score': 0.0,
 'surface_dice': 0.0,
 'voi_score': 0.5894219744166309,
 'voi_split': 6.410649199304792e-17,
 'voi_merge': 2.3219246618108693}

In [ ]:
#T_LOW = 0.30        # Keep connectivity permissive
#T_HIGH = 0.80       # Lowered from 0.85 to find more seeds
#Z_RADIUS = 2        # Increased from 1 to 2 (Fixes vertical splits)
#XY_RADIUS = 1       # Keep 0 (Prevent mergers)
#DUST_MIN_SIZE = 300 # Lowered from 500 to keep smaller valid fragments
#CROP_MARGIN = 9
{'image_score': np.float64(0.6978754393297082),
 'topo_score': 0.4622273249138921,
 'surface_dice': 0.9441961676296846,
 'voi_score': 0.6535388091004308,
 'voi_split': 0.8403331996264144,
 'voi_merge': 0.926770177021606}


#T_LOW = 0.30 
#T_HIGH = 0.80
#Z_RADIUS = 9        # Increased from 1 to 2 (Fixes vertical splits)
#XY_RADIUS = 1       # Keep 0 (Prevent mergers)
#DUST_MIN_SIZE = 300 # Lowered from 500 to keep smaller valid fragments
#CROP_MARGIN = 9
{'image_score': np.float64(0.6993756080223525),
 'topo_score': 0.4672789896670494,
 'surface_dice': 0.9442800188316288,
 'voi_score': 0.653411155803336,
 'voi_split': 0.8405283482404348,
 'voi_merge': 0.9275714723110456}

In [86]:
import os
import numpy as np
from skimage.morphology import skeletonize
from scipy.ndimage import binary_dilation
from scipy import ndimage

def frangi_filter_3d(image, sigmas=(1, 3, 5), beta1=0.5, beta2=15, gamma=None, black_ridges=True):
    """
    Frangi滤波器 - 用于增强3D图像中的管状结构（如血管）。
    
    Frangi滤波器基于Hessian矩阵的特征值来检测管状结构。它通过分析局部二阶导数
    来识别具有"管状"几何特征的区域。
    
    原理：
    1. 计算图像在不同尺度下的Hessian矩阵
    2. 计算Hessian矩阵的特征值（λ1, λ2, λ3）
    3. 根据特征值的关系判断是否为管状结构
    4. 使用Frangi响应函数计算增强值
    
    参数:
        image: numpy数组，形状为 (D, H, W) 或 (D, H, W, 1) 的3D图像
        sigmas: tuple，高斯核的标准差范围，用于多尺度检测
                例如 (1, 3, 5) 表示检测1到5像素宽度的血管
        beta1: float，控制对偏离管状结构的敏感度（默认0.5）
        beta2: float，控制对背景的敏感度（默认15）
        gamma: float，归一化因子，如果为None则自动计算
        black_ridges: bool，True表示检测暗色管状结构（血管），False表示检测亮色结构
    
    返回:
        enhanced: numpy数组，增强后的图像，形状与输入相同
    """
    # 确保输入是3D数组
    if len(image.shape) == 4:
        image = image[..., 0]
    
    # 确保是float类型
    image = image.astype(np.float64)
    
    # 如果检测暗色结构，反转图像
    if black_ridges:
        image = -image
    
    # 初始化输出
    enhanced = np.zeros_like(image)
    
    # 对每个尺度计算Frangi响应
    for sigma in sigmas:
        # 计算Hessian矩阵的各个分量
        # 使用高斯滤波的导数
        hxx = ndimage.gaussian_filter1d(
            ndimage.gaussian_filter1d(
                ndimage.gaussian_filter1d(image, sigma, axis=2, order=2), 
                sigma, axis=1, order=0), 
            sigma, axis=0, order=0)
        
        hyy = ndimage.gaussian_filter1d(
            ndimage.gaussian_filter1d(
                ndimage.gaussian_filter1d(image, sigma, axis=2, order=0), 
                sigma, axis=1, order=2), 
            sigma, axis=0, order=0)
        
        hzz = ndimage.gaussian_filter1d(
            ndimage.gaussian_filter1d(
                ndimage.gaussian_filter1d(image, sigma, axis=2, order=0), 
                sigma, axis=1, order=0), 
            sigma, axis=0, order=2)
        
        hxy = ndimage.gaussian_filter1d(
            ndimage.gaussian_filter1d(
                ndimage.gaussian_filter1d(image, sigma, axis=2, order=1), 
                sigma, axis=1, order=1), 
            sigma, axis=0, order=0)
        
        hxz = ndimage.gaussian_filter1d(
            ndimage.gaussian_filter1d(
                ndimage.gaussian_filter1d(image, sigma, axis=2, order=1), 
                sigma, axis=1, order=0), 
            sigma, axis=0, order=1)
        
        hyz = ndimage.gaussian_filter1d(
            ndimage.gaussian_filter1d(
                ndimage.gaussian_filter1d(image, sigma, axis=2, order=0), 
                sigma, axis=1, order=1), 
            sigma, axis=0, order=1)
        
        # 构建Hessian矩阵并计算特征值
        # 对于3D，我们需要在每个体素处计算3x3矩阵的特征值
        # 这里使用近似方法：计算特征值的平方和
        # 更精确的方法需要逐像素计算特征值，但计算量大
        
        # 简化的Frangi响应计算（基于Hessian矩阵的迹和行列式）
        # 对于管状结构：一个特征值接近0，另外两个较大且符号相同
        trace = hxx + hyy + hzz
        det = (hxx * hyy * hzz + 
               2 * hxy * hxz * hyz - 
               hxx * hyz * hyz - 
               hyy * hxz * hxz - 
               hzz * hxy * hxy)
        
        # Frangi响应函数
        # 对于管状结构，我们希望：
        # - 一个特征值接近0（沿管轴方向）
        # - 另外两个特征值较大且同号（垂直于管轴）
        
        # 使用迹和行列式的组合来近似
        # 归一化
        if gamma is None:
            gamma = np.max(np.abs(trace))
        
        # 计算响应
        # 简化版本：基于Hessian矩阵的Frobenius范数
        hessian_norm = np.sqrt(hxx**2 + hyy**2 + hzz**2 + 
                              2 * (hxy**2 + hxz**2 + hyz**2))
        
        # Frangi响应
        response = np.exp(-beta1 * (trace**2) / (hessian_norm + 1e-10)) * \
                   (1 - np.exp(-beta2 * hessian_norm**2 / (gamma**2 + 1e-10)))
        
        # 只保留正值（管状结构）
        response = np.maximum(response, 0)
        
        # 取所有尺度中的最大值
        enhanced = np.maximum(enhanced, response)
    
    # 归一化到[0, 1]
    if enhanced.max() > 0:
        enhanced = enhanced / enhanced.max()
    
    return enhanced.astype(np.float32)


def apply_frangi_postprocessing(prediction, threshold=0.5, sigmas=(1, 3, 5), 
                                beta1=0.5, beta2=15, use_frangi=True):
    """
    使用Frangi滤波器对分割预测结果进行后处理。
    
    这个方法可以：
    1. 使用Frangi滤波器增强预测结果中的管状结构
    2. 结合原始预测和Frangi增强结果
    3. 应用阈值得到最终的分割掩码
    
    参数:
        prediction: numpy数组，模型预测的概率图，形状为 (D, H, W) 或 (D, H, W, num_classes)
        threshold: float，二值化阈值（默认0.5）
        sigmas: tuple，Frangi滤波器的尺度参数
        beta1: float，Frangi参数
        beta2: float，Frangi参数
        use_frangi: bool，是否使用Frangi滤波器（如果False，只做阈值化）
    
    返回:
        processed_mask: numpy数组，处理后的二值掩码，形状为 (D, H, W)
    """
    # 处理多类别预测
    if len(prediction.shape) == 4:
        # 如果是多类别，取前景类别（假设类别1是血管/墨水）
        if prediction.shape[-1] > 1:
            pred_prob = prediction[..., 1]  # 取类别1的概率
        else:
            pred_prob = prediction[..., 0]
    else:
        pred_prob = prediction
    
    if use_frangi:
        # 应用Frangi滤波器增强管状结构
        frangi_enhanced = frangi_filter_3d(
            pred_prob, 
            sigmas=sigmas, 
            beta1=beta1, 
            beta2=beta2,
            black_ridges=True  # 假设检测暗色结构
        )
        
        # 结合原始预测和Frangi增强结果
        # 方法1：加权平均
        combined = 0.7 * pred_prob + 0.3 * frangi_enhanced
        
        # 方法2：取最大值（保留更强的响应）
        # combined = np.maximum(pred_prob, frangi_enhanced)
        
        # 方法3：只在Frangi检测到的区域增强
        # frangi_mask = frangi_enhanced > 0.3
        # combined = pred_prob.copy()
        # combined[frangi_mask] = np.maximum(pred_prob[frangi_mask], frangi_enhanced[frangi_mask])
        
        processed_prob = combined
    else:
        processed_prob = pred_prob
    
    # 二值化
    processed_mask = (processed_prob > threshold).astype(np.uint8)
    
    return processed_mask

def build_anisotropic_struct(z_radius, xy_radius):
    """
    构建各向异性结构元素用于3D闭运算
    
    参数:
        z_radius: z方向的半径
        xy_radius: xy平面的半径
    
    返回:
        structure: 3D结构元素，如果半径都为0则返回None
    """
    if z_radius == 0 and xy_radius == 0:
        return None
    
    size_z = 2 * z_radius + 1
    size_xy = 2 * xy_radius + 1
    
    struct = np.zeros((size_z, size_xy, size_xy), dtype=bool)
    center_z = z_radius
    center_xy = xy_radius
    
    # 创建各向异性结构
    for z in range(size_z):
        for y in range(size_xy):
            for x in range(size_xy):
                dz = abs(z - center_z)
                dxy = np.sqrt((x - center_xy)**2 + (y - center_xy)**2)
                if dz <= z_radius and dxy <= xy_radius:
                    struct[z, y, x] = True
    
    return struct

In [87]:
def topo_postprocess(
    probs,          # (D, H, W)
    T_low=0.90,
    T_high=0.90,
    z_radius=1,
    xy_radius=0,
    dust_min_size=100,
):
    # --- Step 1: 3D Hysteresis ---
    strong = probs >= T_high
    weak   = probs >= T_low

    if not strong.any():
        return np.zeros_like(probs, dtype=np.uint8)

    struct_hyst = ndi.generate_binary_structure(3, 3)
    mask = ndi.binary_propagation(strong, mask=weak, structure=struct_hyst)

    if not mask.any():
        return np.zeros_like(probs, dtype=np.uint8)

    # --- Step 2: 3D Anisotropic Closing ---
    if z_radius > 0 or xy_radius > 0:
        struct_close = build_anisotropic_struct(z_radius, xy_radius)
        if struct_close is not None:
            mask = ndi.binary_closing(mask, structure=struct_close)

    # --- Step 3: Dust Removal ---
    if dust_min_size > 0:
        mask = remove_small_objects(mask.astype(bool), min_size=dust_min_size)

    return mask.astype(np.uint8)


def topo_postprocess_with_frangi(
    probs,          # (D, H, W)
    T_low=0.90,
    T_high=0.90,
    z_radius=1,
    xy_radius=0,
    dust_min_size=100,
    # Frangi参数
    use_frangi=True,
    frangi_weight=0.3,      # Frangi增强的权重 (0-1)
    frangi_sigmas=(1, 3, 5),
    frangi_beta1=0.5,
    frangi_beta2=15,
    frangi_mode='enhance_before',  # 'enhance_before', 'enhance_after', 'replace_hyst'
):
    """
    集成Frangi滤波器的拓扑后处理
    
    参数:
        probs: 概率图 (D, H, W)
        T_low, T_high: Hysteresis阈值
        z_radius, xy_radius: 各向异性闭运算参数
        dust_min_size: 小对象去除的最小尺寸
        use_frangi: 是否使用Frangi滤波器
        frangi_weight: Frangi增强的权重（0-1之间）
        frangi_sigmas: Frangi滤波器的尺度参数
        frangi_beta1, frangi_beta2: Frangi参数
        frangi_mode: Frangi集成模式
            - 'enhance_before': 在Hysteresis之前增强概率图（推荐）
            - 'enhance_after': 在Hysteresis之后增强掩码
            - 'replace_hyst': 用Frangi增强的概率图替代Hysteresis
    
    返回:
        mask: 处理后的二值掩码 (D, H, W) uint8
    """
    if not use_frangi:
        # 如果不使用Frangi，直接使用原始后处理
        return topo_postprocess(
            probs, T_low, T_high, z_radius, xy_radius, dust_min_size
        )
    
    # 应用Frangi滤波器增强
    frangi_enhanced = frangi_filter_3d(
        probs,
        sigmas=frangi_sigmas,
        beta1=frangi_beta1,
        beta2=frangi_beta2,
        black_ridges=True  # 假设检测暗色结构（血管）
    )
    
    if frangi_mode == 'enhance_before':
        # 模式1: 在Hysteresis之前增强概率图（推荐）
        # 结合原始概率和Frangi增强结果
        enhanced_probs = (1 - frangi_weight) * probs + frangi_weight * frangi_enhanced
        
        # 然后进行标准的拓扑后处理
        return topo_postprocess(
            enhanced_probs, T_low, T_high, z_radius, xy_radius, dust_min_size
        )
    
    elif frangi_mode == 'enhance_after':
        # 模式2: 在Hysteresis之后增强掩码
        # 先进行标准拓扑后处理
        mask = topo_postprocess(
            probs, T_low, T_high, z_radius, xy_radius, dust_min_size
        )
        
        # 将掩码转换为概率图（用于Frangi）
        mask_probs = mask.astype(np.float32)
        
        # 应用Frangi增强
        frangi_mask = frangi_filter_3d(
            mask_probs,
            sigmas=frangi_sigmas,
            beta1=frangi_beta1,
            beta2=frangi_beta2,
            black_ridges=True
        )
        
        # 结合原始掩码和Frangi增强结果
        enhanced_mask = np.maximum(
            mask_probs,
            frangi_mask * frangi_weight
        )
        
        # 二值化
        return (enhanced_mask > 0.5).astype(np.uint8)
    
    elif frangi_mode == 'replace_hyst':
        # 模式3: 用Frangi增强的概率图替代Hysteresis
        # 结合原始概率和Frangi增强结果
        enhanced_probs = (1 - frangi_weight) * probs + frangi_weight * frangi_enhanced
        
        # 使用简单的阈值化替代Hysteresis
        mask = enhanced_probs >= T_high
        
        if not mask.any():
            return np.zeros_like(probs, dtype=np.uint8)
        
        # 然后进行闭运算和小对象去除
        if z_radius > 0 or xy_radius > 0:
            struct_close = build_anisotropic_struct(z_radius, xy_radius)
            if struct_close is not None:
                mask = ndi.binary_closing(mask, structure=struct_close)
        
        if dust_min_size > 0:
            mask = remove_small_objects(mask.astype(bool), min_size=dust_min_size)
        
        return mask.astype(np.uint8)
    
    else:
        raise ValueError(f"Unknown frangi_mode: {frangi_mode}")


In [88]:
prob_pred = sigmoid(logits_pred)
prob_pred = np.transpose(prob_pred.squeeze().cpu().numpy(), (2, 1, 0))

T_low=0.50
T_high=0.90
z_radius=2
xy_radius=0
dust_min_size=100
# Frangi参数
use_frangi=True
frangi_weight=0.3
frangi_sigmas=(1, 3, 5)
frangi_beta1=0.5
frangi_beta2=15
frangi_mode='enhance_after'

CROP_MARGIN = 9
prob_pred[:CROP_MARGIN, :, :] = 0
prob_pred[-CROP_MARGIN:, :, :] = 0
prob_pred[:, :CROP_MARGIN, :] = 0
prob_pred[:, -CROP_MARGIN:, :] = 0
prob_pred[:, :, :CROP_MARGIN] = 0
prob_pred[:, :, -CROP_MARGIN:] = 0

final = topo_postprocess_with_frangi(
        prob_pred,
        T_low=T_low,
        T_high=T_high,
        z_radius=z_radius,
        xy_radius=xy_radius,
        dust_min_size=dust_min_size,
        use_frangi=use_frangi,
        frangi_weight=frangi_weight,
        frangi_sigmas=frangi_sigmas,
        frangi_beta1=frangi_beta1,
        frangi_beta2=frangi_beta2,
        frangi_mode=frangi_mode,
    )

In [89]:
final_mask = final
#3545147380

In [ ]:
out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/final_pred_3545147380_2.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/3545147380.tif"
tifffile.imwrite(out_path, final_mask)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_3545147380_2.tif")

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report
# origin 0.5305181364493412	0.07948210753461608	0.8984319807505937	0.5492066026464245

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_3545147380_2.tif
Done!


{'image_score': np.float64(0.5336816641958646),
 'topo_score': 0.11994572591587517,
 'surface_dice': 0.8763440854818639,
 'voi_score': 0.545650047149856,
 'voi_split': 1.1428135789370542,
 'voi_merge': 1.6327746405538806}

In [ ]:
#CROP_MARGIN = 13


#CROP_MARGIN = 9
{'image_score': np.float64(0.6928263230558964),
 'topo_score': 0.445279804451687,
 'surface_dice': 0.9405297829991504,
 'voi_score': 0.6573055933448219,
 'voi_split': 0.848328060203944,
 'voi_merge': 0.8895465302703475}

# baseline
{'image_score': np.float64(0.6104108550613401),
 'topo_score': 0.1529262777023971,
 'surface_dice': 0.9510553115288458,
 'voi_score': 0.6618960363300709,
 'voi_split': 0.8085355515981472,
 'voi_merge': 0.8941687257713037}

In [ ]:
# Remover os cantos à mão não resolveu nada
import nibabel as nib

prediction_post_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_2536049117_2.nii.gz-label.nii.gz"
# Load nifti
nifti_img = nib.load(prediction_post_path)

# 2. Get the actual data as a NumPy array
prediction_post = nifti_img.get_fdata().astype(np.int16)
prediction_post[prediction_post>0.5]=1

#prediction_post = np.transpose(prediction_post, (2, 1, 0))
#prediction_post = np.transpose(prediction_post, (2, 0, 1))
#prediction_post = np.transpose(prediction_post, (0, 2, 1))
#prediction_post = np.transpose(prediction_post, (2, 1, 0))
#prediction_post = np.flip(prediction_post, axis=1)
prediction_post = np.flip(prediction_post, axis=1)
prediction_post = np.flip(prediction_post, axis=0) 
prediction_post = np.transpose(prediction_post, (2, 1, 0)).astype(np.uint16)



out_path = f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/mine_final_pred_2536049117_2.tif"
gt_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/DataSet/val_labels/2536049117.tif"
tifffile.imwrite(out_path, prediction_post)

split_by_object(out_path, f"/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_2536049117_2.tif")

score_report = score_single_tif(
    gt_path=gt_path,
    pred_path=out_path,
    surface_tolerance=2.0,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
)
score_report

Loading image...
Saving to /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/trash/post_process_5/categorical_final_pred_2536049117_2.tif
Done!


{'image_score': np.float64(0.6780419710157646),
 'topo_score': 0.4046731857951621,
 'surface_dice': 0.9347838644857932,
 'voi_score': 0.6556161791633951,
 'voi_split': 0.859161504979191,
 'voi_merge': 0.8917807474225056}